In [1]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [3]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
from ray.tune.registry import register_env
from py_dss_interface import DSSDLL
import pandas as pd
import torch
import warnings
import logging
import csv

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Configure logging
logging.basicConfig(level=logging.INFO)  # Change to DEBUG for more detailed logs
logger = logging.getLogger(__name__)

# Determine device for PyTorch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set a seed for reproducibility
SEED = 42

# Define lengths for each segment of the action space
PV_KVAR_ACTION_LEN = 30          # Number of PV systems for reactive power control
PV_KW_ACTION_LEN = 30            # Number of PV systems for active power control
BESS_KW_ACTION_LEN = 30          # Number of Batteries for active power control
TRANSFORMER_TAPS_ACTION_LEN = 1  # Number of transformer taps
CAPACITOR_ACTION_LEN = 2         # Number of Capacitors (Manually defined)

# Define action value ranges for each controllable element
PV_KVAR_ACTION_LOW = -80
PV_KVAR_ACTION_HIGH = 80
PV_KW_ACTION_LOW = 0
PV_KW_ACTION_HIGH = 100
BESS_KW_ACTION_LOW = 0
BESS_KW_ACTION_HIGH = 100
TRANSFORMER_TAPS_ACTION_LOW = 0.9
TRANSFORMER_TAPS_ACTION_HIGH = 1.1
CAPACITOR_ACTION_LOW = 0
CAPACITOR_ACTION_HIGH = 1

class PowerSystemEnv(gym.Env):
    """
    Custom Environment for Power System Control using OpenDSS and Gymnasium.
    """
    def __init__(self, dss_path, dss_file, irradiance_csv_file, load_profile_file, load_powers_file, seed=None, load_scale=3):
        super(PowerSystemEnv, self).__init__()
        self.seed(seed)
        
        # Initialize OpenDSS Controller
        self.controller = DSSDLL(dss_path)
        self.controller.text(f"compile [{dss_file}]")
        logger.info(f"[INIT] Compiled OpenDSS file: {dss_file}")
        
        # Manually define capacitor names
        self.capacitor_names = ["CAP_201", "CAP_301"]
        logger.info(f"[INIT] Defined capacitors: {self.capacitor_names}")
        
        # Retrieve bus names from OpenDSS
        self.ranked_buses = self.get_bus_names()
        logger.info(f"[INIT] Retrieved {len(self.ranked_buses)} buses from OpenDSS.")
        
          # Manually define load names
        self.load_names = [
            "Load_1003", "Load_1004", "Load_1005", "Load_1006", "Load_1007", "Load_1008",
            "Load_1009", "Load_1010", "Load_1011", "Load_1012", "Load_1013", "Load_1014",
            "Load_1015", "Load_1016", "Load_1017", "Load_2002", "Load_2003", "Load_2005",
            "Load_2008", "Load_2009", "Load_2010", "Load_2011", "Load_2014", "Load_2015",
            "Load_2016", "Load_2017", "Load_2018", "Load_2020", "Load_2022", "Load_2023",
            "Load_2024", "Load_2025", "Load_2028", "Load_2029", "Load_2030", "Load_2031",
            "Load_2032", "Load_2034", "Load_2035", "Load_2037", "Load_2040", "Load_2041",
            "Load_2042", "Load_2043", "Load_2045", "Load_2046", "Load_2047", "Load_2048",
            "Load_2049", "Load_2050", "Load_2051", "Load_2052", "Load_2053", "Load_2054",
            "Load_2055", "Load_2056", "Load_2058", "Load_2059", "Load_2060",
            "Load_3002", "Load_3004", "Load_3006", "Load_3007", "Load_3009", "Load_3010",
            "Load_3011", "Load_3012", "Load_3013", "Load_3014", "Load_3016", "Load_3017",
            "Load_3018", "Load_3019", "Load_3020", "Load_3021", "Load_3023", "Load_3024",
            "Load_3025", "Load_3026", "Load_3027", "Load_3028", "Load_3029", "Load_3031",
            "Load_3032", "Load_3033", "Load_3034", "Load_3035", "Load_3036", "Load_3037",
            "Load_3038", "Load_3039", "Load_3041", "Load_3042", "Load_3043", "Load_3044",
            "Load_3045", "Load_3047", "Load_3048", "Load_3049", "Load_3050", "Load_3051",
            "Load_3052", "Load_3054", "Load_3056", "Load_3057", "Load_3058", "Load_3059",
            "Load_3060", "Load_3061", "Load_3062", "Load_3063", "Load_3064", "Load_3065",
            "Load_3066", "Load_3067", "Load_3070", "Load_3071", "Load_3072", "Load_3073",
            "Load_3074", "Load_3077", "Load_3078", "Load_3081", "Load_3083", "Load_3084",
            "Load_3085", "Load_3086", "Load_3087", "Load_3088", "Load_3089", "Load_3090",
            "Load_3091", "Load_3093", "Load_3094", "Load_3095", "Load_3096", "Load_3097",
            "Load_3098", "Load_3099", "Load_3101", "Load_3102", "Load_3103", "Load_3104",
            "Load_3105", "Load_3106", "Load_3108", "Load_3109", "Load_3110", "Load_3111",
            "Load_3112", "Load_3114", "Load_3115", "Load_3116", "Load_3117", "Load_3120",
            "Load_3121", "Load_3122", "Load_3123", "Load_3124", "Load_3125", "Load_3126",
            "Load_3127", "Load_3128", "Load_3129", "Load_3130", "Load_3131", "Load_3132",
            "Load_3134", "Load_3135", "Load_3136", "Load_3137", "Load_3138", "Load_3139",
            "Load_3141", "Load_3142", "Load_3143", "Load_3144", "Load_3145", "Load_3146",
            "Load_3147", "Load_3148", "Load_3149", "Load_3150", "Load_3151", "Load_3152",
            "Load_3153", "Load_3154", "Load_3155", "Load_3157", "Load_3158", "Load_3159",
            "Load_3160", "Load_3161", "Load_3162"
        ]
        logger.info(f"[INIT] Defined loads: {len(self.load_names)} loads.")
        
        # Load profiles
        self.irradiance_profile = self.load_csv(irradiance_csv_file, "Irradiance")
        self.load_profile = self.load_csv(load_profile_file, "Load Profile", skip_header=True)
        self.load_powers = self.load_csv(load_powers_file, "Load Powers", skip_header=False)
        
        # Validate load powers
        if len(self.load_powers) != len(self.load_names):
            raise ValueError("Number of load powers does not match number of load names.")
        logger.info("[INIT] Load powers and load names are consistent.")
        
        # Define action and observation spaces
        self.action_space = spaces.Box(
            low=np.array(
                [PV_KVAR_ACTION_LOW] * PV_KVAR_ACTION_LEN +
                [PV_KW_ACTION_LOW] * PV_KW_ACTION_LEN +
                [BESS_KW_ACTION_LOW] * BESS_KW_ACTION_LEN +
                [TRANSFORMER_TAPS_ACTION_LOW] * TRANSFORMER_TAPS_ACTION_LEN +
                [CAPACITOR_ACTION_LOW] * CAPACITOR_ACTION_LEN
            ),
            high=np.array(
                [PV_KVAR_ACTION_HIGH] * PV_KVAR_ACTION_LEN +
                [PV_KW_ACTION_HIGH] * PV_KW_ACTION_LEN +
                [BESS_KW_ACTION_HIGH] * BESS_KW_ACTION_LEN +
                [TRANSFORMER_TAPS_ACTION_HIGH] * TRANSFORMER_TAPS_ACTION_LEN +
                [CAPACITOR_ACTION_HIGH] * CAPACITOR_ACTION_LEN
            ),
            dtype=np.float32
        )
        logger.info(f"[INIT] Defined action space with shape: {self.action_space.shape}")
        
        num_buses = len(self.ranked_buses)
        self.observation_space = spaces.Box(
            low=0,
            high=2.0,
            shape=(num_buses,),
            dtype=np.float32
        )
        logger.info(f"[INIT] Defined observation space with shape: {self.observation_space.shape}")
        
        # Initialize other variables
        self.current_step = 0
        self.control_steps = 0
        self.max_control_steps = 0  # Define as needed
        self.hourly_violations_count = []
        self.all_bus_voltages = []
    
    def load_csv(self, file_path, description, skip_header=False):
        """
        Utility method to load CSV files.
        """
        try:
            with open(file_path, 'r') as csvfile:
                reader = csv.reader(csvfile)
                if skip_header:
                    next(reader, None)  # Skip header if present
                data = [float(row[0]) for row in reader]
            logger.info(f"[LOAD] Loaded {description} with {len(data)} entries.")
            return data
        except Exception as e:
            logger.error(f"[LOAD] Failed to load {description} from {file_path}: {e}")
            raise e
    
    def seed(self, seed=None):
        """
        Set the seed for reproducibility.
        """
        self.seed_value = seed
        np.random.seed(seed)
        torch.manual_seed(seed)
        if seed is not None:
            torch.cuda.manual_seed_all(seed)
        logger.info(f"[SEED] Environment seeded with value: {seed}")
    
    def get_bus_names(self):
        """
        Retrieve all bus names from OpenDSS.
        """
        bus_names = self.controller.circuit_all_bus_names()
        if isinstance(bus_names, list):
            logger.info(f"[INFO] Retrieved {len(bus_names)} bus names.")
            return bus_names
        else:
            raise TypeError(f"Expected list for bus names, got {type(bus_names)}")
    
    def _take_action(self, action):
        """
        Apply the given action to the OpenDSS model.
        """
        # Split action into respective components
        pv_kvar = action[:PV_KVAR_ACTION_LEN].copy()
        pv_kw = action[PV_KVAR_ACTION_LEN:PV_KVAR_ACTION_LEN + PV_KW_ACTION_LEN].copy()
        battery_kw = action[PV_KVAR_ACTION_LEN + PV_KW_ACTION_LEN:
                            PV_KVAR_ACTION_LEN + PV_KW_ACTION_LEN + BESS_KW_ACTION_LEN].copy()
        transformer_tap = action[PV_KVAR_ACTION_LEN + PV_KW_ACTION_LEN + BESS_KW_ACTION_LEN].copy()
        capacitor_states = action[PV_KVAR_ACTION_LEN + PV_KW_ACTION_LEN + BESS_KW_ACTION_LEN + TRANSFORMER_TAPS_ACTION_LEN:].copy()
        capacitor_states = np.round(capacitor_states)
        
        # Debugging: Print actions
        logger.debug(f"[ACTION] Applying Actions:")
        logger.debug(f"  PV_kVAR Actions: {pv_kvar}")
        logger.debug(f"  PV_kW Actions: {pv_kw}")
        logger.debug(f"  Battery_kW Actions: {battery_kw}")
        logger.debug(f"  Transformer Tap Action: {transformer_tap}")
        logger.debug(f"  Capacitor States: {capacitor_states}")
        
        # Retrieve current irradiance
        irradiance = self.irradiance_profile[self.current_step % len(self.irradiance_profile)]
        
        # Clip PV_kW based on irradiance
        for z in range(PV_KW_ACTION_LEN):
            if pv_kw[z] > irradiance * self.KWrated:
                pv_kw[z] = irradiance * self.KWrated
        
        # Clip PV_kVAR based on maximum apparent power (S_max)
        S_max = self.KWrated  # Assuming KWrated represents S_max
        q_max1 = np.sqrt(S_max**2 - np.power(pv_kw, 2))
        pv_kvar = np.clip(pv_kvar, -q_max1, q_max1)
        logger.debug(f"[ACTION] PV_kVAR after clipping: {pv_kvar}")
        
        # Apply PV and Battery actions
        for i in range(min(PV_KW_ACTION_LEN, len(self.ranked_buses))):
            bus = self.ranked_buses[i]
            scaled_pv_kw = pv_kw[i]
            self.controller.text(f"edit PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR={pv_kvar[i]} Pmpp={scaled_pv_kw}")
            logger.debug(f"[CONTROL] Edited PVSystem.PV{i+1} on Bus {bus}: kVAR={pv_kvar[i]}, Pmpp={scaled_pv_kw}")
            
            self.controller.text(f"edit Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW={battery_kw[i]} kVAR=0")
            logger.debug(f"[CONTROL] Edited Storage.Battery{i+1} on Bus {bus}: kW={battery_kw[i]}, kVAR=0")
        
        # Apply Transformer Tap action
        self.controller.text(f"edit Transformer.reg1a taps={transformer_tap}")
        logger.debug(f"[CONTROL] Edited Transformer.reg1a: taps={transformer_tap}")
        
        # Apply Capacitor States
        for i, cap_name in enumerate(self.capacitor_names):
            if i < len(capacitor_states):
                state = int(capacitor_states[i])
                self.controller.text(f"edit Capacitor.{cap_name} states={state}")
                logger.debug(f"[CONTROL] Edited Capacitor.{cap_name}: states={state}")
        
        # Solve the power flow
        self.controller.text("set controlmode=off")
        self.controller.text("solve")
        logger.debug(f"[ACTION] Actions applied and OpenDSS solved.")
    
    def step(self, action):
        """
        Perform a single step in the environment.
        """
        logger.info(f"[STEP] Step {self.current_step}: Taking action.")
        self._take_action(action)
        
        # Retrieve bus voltages
        all_bus_voltages = self.controller.circuit_all_bus_vmag_pu()
        if isinstance(all_bus_voltages, list):
            pass  # Already a list of floats
        elif isinstance(all_bus_voltages, str):
            all_bus_voltages = [float(v) for v in all_bus_voltages.split()]
        else:
            raise TypeError(f"Unexpected type for bus voltages: {type(all_bus_voltages)}")
        
        # Calculate voltage violations
        voltage_violations = sum(1 for v in all_bus_voltages if (v <= 0.95 or v >= 1.05) and v != 0)
        logger.debug(f"[OBSERVATION] Bus Voltages: {all_bus_voltages}")
        logger.info(f"[REWARD] Voltage Violations: {voltage_violations}")
        
        # Define reward
        reward = -voltage_violations
        
        # Update logs
        self.hourly_violations_count.append(voltage_violations)
        self.all_bus_voltages.append(all_bus_voltages)
        
        # Get observation
        observation = self.get_observation()
        
        # Determine if the episode is done
        termination = (self.control_steps >= self.max_control_steps) or (voltage_violations == 0)
        self.control_steps += 1
        
        if termination:
            logger.info(f"[TERMINATION] Episode terminated at step {self.control_steps} with reward {reward}.")
        
        # Return observation, reward, done, truncated, info
        return observation, reward, termination, False, {}
    
    def reset(self, *, seed=None, options=None):
        """
        Reset the environment to an initial state.
        """
        if seed is not None:
            np.random.seed(seed)
        if options is not None:
            logger.info(f"[RESET] Reset options: {options}")
        
        # Recompile OpenDSS to reset the state
        self.controller.text(f"compile [{self.controller.get_dss_file_path()}]")
        logger.info(f"[RESET] Recompiled OpenDSS file: {self.controller.get_dss_file_path()}")
        
        # Update the current step
        self.current_step = (self.current_step + 1) % len(self.irradiance_profile)
        self.control_steps = 0
        
        # Apply load scales
        load_scales = np.random.normal(loc=self.load_profile[self.current_step], scale=0.001, size=len(self.load_names))
        logger.info(f"[RESET] Applying Load Scales for step {self.current_step}.")
        
        for load_name, load_scale in zip(self.load_names, load_scales):
            load_power = self.load_powers[self.load_names.index(load_name)]
            result = load_power * load_scale * self.load_scale
            self.controller.text(f"edit Load.{load_name} kW={result}")
            logger.debug(f"[LOAD] Edited Load.{load_name}: kW={result}")
        
        # Apply PV and Battery initial states based on irradiance
        irradiance = self.irradiance_profile[self.current_step]
        logger.info(f"[RESET] Current Irradiance: {irradiance}")
        
        for i in range(min(PV_KW_ACTION_LEN, len(self.ranked_buses))):
            bus = self.ranked_buses[i]
            fixed_power_factor = 0.9
            pv_kw = 100 * irradiance
            power_factor_angle = np.arccos(fixed_power_factor)
            pv_kvar = pv_kw * np.tan(power_factor_angle)
            battery_kw = self.load_profile[self.current_step] * 100 * self.load_scale
            self.controller.text(f"edit PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR={pv_kvar} Pmpp={pv_kw}")
            logger.debug(f"[LOAD] Edited PVSystem.PV{i+1} on Bus {bus}: kVAR={pv_kvar}, Pmpp={pv_kw}")
            
            self.controller.text(f"edit Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW={battery_kw} kVAR=0")
            logger.debug(f"[LOAD] Edited Storage.Battery{i+1} on Bus {bus}: kW={battery_kw}, kVAR=0")
        
        # Solve the power flow after resetting
        self.controller.text("set controlmode=off")
        self.controller.text("solve")
        logger.info(f"[RESET] Load and PV/Battery states updated and OpenDSS solved.")
        
        # Return the initial observation
        return self.get_observation(), {}
    
    def get_observation(self):
        """
        Retrieve the current observation (bus voltages).
        """
        all_bus_voltages = self.controller.circuit_all_bus_vmag_pu()
        if isinstance(all_bus_voltages, list):
            pass  # Already a list of floats
        elif isinstance(all_bus_voltages, str):
            all_bus_voltages = [float(v) for v in all_bus_voltages.split()]
        else:
            raise TypeError(f"Unexpected type for bus voltages: {type(all_bus_voltages)}")
        
        logger.debug(f"[GET_OBSERVATION] Retrieved Bus Voltages: {all_bus_voltages}")
        return np.array(all_bus_voltages).flatten()

# Define the paths to your data files
dss_path = r"C:\Program Files\OpenDSS"  # Update if different
dss_file = r"C:\Users\Alaa\Documents\OpenDSS Model\OpenDSS Model (08.13.2020)\Master.dss"
irradiance_csv_file = r"D:\Alaa_Selim\Irradiance_Profile_Santa_Clara.csv"
load_profile_file = r"D:\Alaa_Selim\LoadShape1.csv"
load_powers_file = r"C:\Users\Alaa\Documents\OpenDSS Model\OpenDSS Model (08.13.2020)\Loaddemand.csv"

def env_creator(env_config):
    """
    Environment creator for registering with RLlib.
    """
    return PowerSystemEnv(
        dss_path=dss_path,
        dss_file=dss_file,
        irradiance_csv_file=irradiance_csv_file,
        load_profile_file=load_profile_file,
        load_powers_file=load_powers_file,
        seed=SEED,
        load_scale=1.75  # Adjust based on your system requirements
    )

# Register the environment with RLlib
register_env("PowerSystemEnv", env_creator)


In [1]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
from ray.tune.registry import register_env
from py_dss_interface import DSSDLL
import pandas as pd
import torch
import warnings
import logging
import csv

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Configure logging
logging.basicConfig(level=logging.DEBUG)  # Set to DEBUG for detailed logs
logger = logging.getLogger(__name__)

# Determine device for PyTorch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set a seed for reproducibility
SEED = 42

# Define lengths for each segment of the action space
PV_KVAR_ACTION_LEN = 30          # Number of PV systems for reactive power control
PV_KW_ACTION_LEN = 30            # Number of PV systems for active power control
BESS_KW_ACTION_LEN = 30          # Number of Batteries for active power control
TRANSFORMER_TAPS_ACTION_LEN = 1  # Number of transformer taps
CAPACITOR_ACTION_LEN = 2         # Number of Capacitors (Manually defined)

# Define action value ranges for each controllable element
PV_KVAR_ACTION_LOW = -800
PV_KVAR_ACTION_HIGH = 800
PV_KW_ACTION_LOW = 0
PV_KW_ACTION_HIGH = 1000
BESS_KW_ACTION_LOW = 0
BESS_KW_ACTION_HIGH = 1000
TRANSFORMER_TAPS_ACTION_LOW = 0.9
TRANSFORMER_TAPS_ACTION_HIGH = 1.1
CAPACITOR_ACTION_LOW = 0
CAPACITOR_ACTION_HIGH = 1

class PowerSystemEnv(gym.Env):
    """
    Custom Environment for Power System Control using OpenDSS and Gymnasium.
    """
    metadata = {'render.modes': ['human'], 'spec.max_episode_steps': 1000}

    def __init__(self, dss_path, dss_file, irradiance_csv_file, load_profile_file, load_powers_file, seed=None, load_scale=3):
        super(PowerSystemEnv, self).__init__()
        self.seed(seed)
        
        # Store the OpenDSS file path for later use
        self.dss_file = dss_file
        
        # Initialize OpenDSS Controller
        self.controller = DSSDLL(dss_path)
        self.controller.text(f"compile [{dss_file}]")
        logger.info(f"[INIT] Compiled OpenDSS file: {dss_file}")
        
        # Manually define capacitor names
        self.capacitor_names = ["CAP_201", "CAP_301"]
        logger.info(f"[INIT] Defined capacitors: {self.capacitor_names}")
        
        # Retrieve bus names from OpenDSS
        self.ranked_buses = self.get_bus_names()
        logger.info(f"[INIT] Retrieved {len(self.ranked_buses)} buses from OpenDSS.")
        
        # Manually define load names
        self.load_names = [
            "Load_1003", "Load_1004", "Load_1005", "Load_1006", "Load_1007", "Load_1008",
            "Load_1009", "Load_1010", "Load_1011", "Load_1012", "Load_1013", "Load_1014",
            "Load_1015", "Load_1016", "Load_1017", "Load_2002", "Load_2003", "Load_2005",
            "Load_2008", "Load_2009", "Load_2010", "Load_2011", "Load_2014", "Load_2015",
            "Load_2016", "Load_2017", "Load_2018", "Load_2020", "Load_2022", "Load_2023",
            "Load_2024", "Load_2025", "Load_2028", "Load_2029", "Load_2030", "Load_2031",
            "Load_2032", "Load_2034", "Load_2035", "Load_2037", "Load_2040", "Load_2041",
            "Load_2042", "Load_2043", "Load_2045", "Load_2046", "Load_2047", "Load_2048",
            "Load_2049", "Load_2050", "Load_2051", "Load_2052", "Load_2053", "Load_2054",
            "Load_2055", "Load_2056", "Load_2058", "Load_2059", "Load_2060",
            "Load_3002", "Load_3004", "Load_3006", "Load_3007", "Load_3009", "Load_3010",
            "Load_3011", "Load_3012", "Load_3013", "Load_3014", "Load_3016", "Load_3017",
            "Load_3018", "Load_3019", "Load_3020", "Load_3021", "Load_3023", "Load_3024",
            "Load_3025", "Load_3026", "Load_3027", "Load_3028", "Load_3029", "Load_3031",
            "Load_3032", "Load_3033", "Load_3034", "Load_3035", "Load_3036", "Load_3037",
            "Load_3038", "Load_3039", "Load_3041", "Load_3042", "Load_3043", "Load_3044",
            "Load_3045", "Load_3047", "Load_3048", "Load_3049", "Load_3050", "Load_3051",
            "Load_3052", "Load_3054", "Load_3056", "Load_3057", "Load_3058", "Load_3059",
            "Load_3060", "Load_3061", "Load_3062", "Load_3063", "Load_3064", "Load_3065",
            "Load_3066", "Load_3067", "Load_3070", "Load_3071", "Load_3072", "Load_3073",
            "Load_3074", "Load_3077", "Load_3078", "Load_3081", "Load_3083", "Load_3084",
            "Load_3085", "Load_3086", "Load_3087", "Load_3088", "Load_3089", "Load_3090",
            "Load_3091", "Load_3093", "Load_3094", "Load_3095", "Load_3096", "Load_3097",
            "Load_3098", "Load_3099", "Load_3101", "Load_3102", "Load_3103", "Load_3104",
            "Load_3105", "Load_3106", "Load_3108", "Load_3109", "Load_3110", "Load_3111",
            "Load_3112", "Load_3114", "Load_3115", "Load_3116", "Load_3117", "Load_3120",
            "Load_3121", "Load_3122", "Load_3123", "Load_3124", "Load_3125", "Load_3126",
            "Load_3127", "Load_3128", "Load_3129", "Load_3130", "Load_3131", "Load_3132",
            "Load_3134", "Load_3135", "Load_3136", "Load_3137", "Load_3138", "Load_3139",
            "Load_3141", "Load_3142", "Load_3143", "Load_3144", "Load_3145", "Load_3146",
            "Load_3147", "Load_3148", "Load_3149", "Load_3150", "Load_3151", "Load_3152",
            "Load_3153", "Load_3154", "Load_3155", "Load_3157", "Load_3158", "Load_3159",
            "Load_3160", "Load_3161", "Load_3162"
        ]
        logger.info(f"[INIT] Defined loads: {len(self.load_names)} loads loaded.")
        
        # Validate load names against OpenDSS
        self.validate_loads()
        
        # Define system parameters
        self.KWrated = 1000
        self.load_scale = load_scale
        
        # Initialize PV systems and Batteries
        num_pv = PV_KW_ACTION_LEN  # 30 PV systems
        num_battery = BESS_KW_ACTION_LEN  # 30 Batteries
        logger.info(f"[INIT] Initializing {num_pv} PV systems and {num_battery} Batteries.")
        
        for i in range(num_pv):
            if i < len(self.ranked_buses):
                bus = self.ranked_buses[i]
                self.controller.text(f"new PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR=0 KVA=100 Pmpp=80")
                logger.debug(f"[INIT] Initialized PVSystem.PV{i+1} on Bus {bus}")
            else:
                logger.warning(f"[INIT] Not enough buses to initialize PVSystem.PV{i+1}")
        
        for i in range(num_battery):
            if i < len(self.ranked_buses):
                bus = self.ranked_buses[i]
                self.controller.text(f"new Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW=100 kVAR=0")
                logger.debug(f"[INIT] Initialized Storage.Battery{i+1} on Bus {bus}")
            else:
                logger.warning(f"[INIT] Not enough buses to initialize Storage.Battery{i+1}")
        
        # Load profiles
        self.irradiance_profile = self.load_csv(irradiance_csv_file, "Irradiance")
        self.load_profile = self.load_csv(load_profile_file, "Load Profile", skip_header=True)
        self.load_powers = self.load_csv(load_powers_file, "Load Powers", skip_header=False)
        
        # Validate load powers
        if len(self.load_powers) != len(self.load_names):
            raise ValueError("Number of load powers does not match number of load names.")
        logger.info("[INIT] Load powers and load names are consistent.")
        
        # Define action and observation spaces
        self.action_space = spaces.Box(
            low=np.array(
                [PV_KVAR_ACTION_LOW] * PV_KVAR_ACTION_LEN +
                [PV_KW_ACTION_LOW] * PV_KW_ACTION_LEN +
                [BESS_KW_ACTION_LOW] * BESS_KW_ACTION_LEN +
                [TRANSFORMER_TAPS_ACTION_LOW] * TRANSFORMER_TAPS_ACTION_LEN +
                [CAPACITOR_ACTION_LOW] * CAPACITOR_ACTION_LEN
            ),
            high=np.array(
                [PV_KVAR_ACTION_HIGH] * PV_KVAR_ACTION_LEN +
                [PV_KW_ACTION_HIGH] * PV_KW_ACTION_LEN +
                [BESS_KW_ACTION_HIGH] * BESS_KW_ACTION_LEN +
                [TRANSFORMER_TAPS_ACTION_HIGH] * TRANSFORMER_TAPS_ACTION_LEN +
                [CAPACITOR_ACTION_HIGH] * CAPACITOR_ACTION_LEN
            ),
            dtype=np.float32
        )
        logger.info(f"[INIT] Defined action space with shape: {self.action_space.shape}")
        
        # Update the number of observations based on current setup
        num_buses = len(self.ranked_buses)
        expected_observations = 915
        if num_buses != expected_observations:
            logger.warning(f"[INIT] Number of buses ({num_buses}) does not match expected observations ({expected_observations}). Please verify.")
        
        self.observation_space = spaces.Box(
            low=0.0,
            high=2.0,
            shape=(expected_observations,),
            dtype=np.float32
        )
        logger.info(f"[INIT] Defined observation space with shape: {self.observation_space.shape}")
        
        # Initialize other variables
        self.current_step = 0
        self.control_steps = 0
        self.max_control_steps = 3  # Define as needed
        self.hourly_violations_count = []
        self.all_bus_voltages = []
    
    def load_csv(self, file_path, description, skip_header=False):
        """
        Utility method to load CSV files.
        """
        try:
            with open(file_path, 'r') as csvfile:
                reader = csv.reader(csvfile)
                if skip_header:
                    next(reader, None)  # Skip header if present
                data = [float(row[0]) for row in reader]
            logger.info(f"[LOAD] Loaded {description} with {len(data)} entries.")
            return data
        except Exception as e:
            logger.error(f"[LOAD] Failed to load {description} from {file_path}: {e}")
            raise e
    
    def validate_loads(self):
        """
        Validates that all load names exist in the OpenDSS model.
        """
        # Since 'circuit_all_load_names()' is unavailable, manual validation is limited.
        # Ensure that the load names in self.load_names match those in OpenDSS.
        # Alternatively, proceed with caution.
        logger.info("[VALIDATE] Manual validation of load names is required.")
    
    def seed(self, seed=None):
        """
        Set the seed for reproducibility.
        """
        self.seed_value = seed
        np.random.seed(seed)
        torch.manual_seed(seed)
        if seed is not None:
            torch.cuda.manual_seed_all(seed)
        logger.info(f"[SEED] Environment seeded with value: {seed}")
    
    def get_bus_names(self):
        """
        Retrieve all bus names from OpenDSS.
        """
        bus_names = self.controller.circuit_all_bus_names()
        if isinstance(bus_names, list):
            logger.info(f"[INFO] Retrieved {len(bus_names)} bus names.")
            return bus_names
        else:
            raise TypeError(f"Expected list for bus names, got {type(bus_names)}")
    
    def _take_action(self, action):
        """
        Apply the given action to the OpenDSS model.
        """
        # Split action into respective components
        pv_kvar = action[:PV_KVAR_ACTION_LEN].copy()
        pv_kw = action[PV_KVAR_ACTION_LEN:PV_KVAR_ACTION_LEN + PV_KW_ACTION_LEN].copy()
        battery_kw = action[PV_KVAR_ACTION_LEN + PV_KW_ACTION_LEN:
                            PV_KVAR_ACTION_LEN + PV_KW_ACTION_LEN + BESS_KW_ACTION_LEN].copy()
        transformer_tap = action[PV_KVAR_ACTION_LEN + PV_KW_ACTION_LEN + BESS_KW_ACTION_LEN].copy()
        capacitor_states = action[PV_KVAR_ACTION_LEN + PV_KW_ACTION_LEN + BESS_KW_ACTION_LEN + TRANSFORMER_TAPS_ACTION_LEN:].copy()
        capacitor_states = np.round(capacitor_states)
        
        # Debugging: Print actions
        logger.debug(f"[ACTION] Applying Actions:")
        logger.debug(f"  PV_kVAR Actions: {pv_kvar}")
        logger.debug(f"  PV_kW Actions: {pv_kw}")
        logger.debug(f"  Battery_kW Actions: {battery_kw}")
        logger.debug(f"  Transformer Tap Action: {transformer_tap}")
        logger.debug(f"  Capacitor States: {capacitor_states}")
        
        # Retrieve current irradiance
        irradiance = self.irradiance_profile[self.current_step % len(self.irradiance_profile)]
        logger.debug(f"[INFO] Current Irradiance: {irradiance}")
        
        # Clip PV_kW based on irradiance
        for z in range(PV_KW_ACTION_LEN):
            if pv_kw[z] > irradiance * self.KWrated:
                logger.debug(f"[CLIP] PV_kW before clipping: {pv_kw[z]} > Irradiance * KWrated: {irradiance * self.KWrated}")
                pv_kw[z] = irradiance * self.KWrated
                logger.debug(f"[CLIP] PV_kW after clipping: {pv_kw[z]}")
        
        # Clip PV_kVAR based on maximum apparent power (S_max)
        S_max = self.KWrated  # Assuming KWrated represents S_max
        q_max1 = np.sqrt(S_max**2 - np.power(pv_kw, 2))
        pv_kvar = np.clip(pv_kvar, -q_max1, q_max1)
        logger.debug(f"[ACTION] PV_kVAR after clipping: {pv_kvar}")
        
        # Apply PV and Battery actions
        for i in range(min(PV_KW_ACTION_LEN, len(self.ranked_buses))):
            bus = self.ranked_buses[i]
            scaled_pv_kw = pv_kw[i]
            self.controller.text(f"edit PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR={pv_kvar[i]} Pmpp={scaled_pv_kw}")
            logger.debug(f"[CONTROL] Edited PVSystem.PV{i+1} on Bus {bus}: kVAR={pv_kvar[i]}, Pmpp={scaled_pv_kw}")
            
            self.controller.text(f"edit Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW={battery_kw[i]} kVAR=0")
            logger.debug(f"[CONTROL] Edited Storage.Battery{i+1} on Bus {bus}: kW={battery_kw[i]}, kVAR=0")
        
        # Apply Transformer Tap action
        self.controller.text(f"edit Transformer.reg1a taps={transformer_tap}")
        logger.debug(f"[CONTROL] Edited Transformer.reg1a: taps={transformer_tap}")
        
        # Apply Capacitor States
        for i, cap_name in enumerate(self.capacitor_names):
            if i < len(capacitor_states):
                state = int(capacitor_states[i])
                self.controller.text(f"edit Capacitor.{cap_name} states={state}")
                logger.debug(f"[CONTROL] Edited Capacitor.{cap_name}: states={state}")
        
        # Solve the power flow
        self.controller.text("set controlmode=off")
        self.controller.text("solve")
        logger.debug(f"[ACTION] Actions applied and OpenDSS solved.")
    
    def step(self, action):
        """
        Perform a single step in the environment.
        """
        logger.info(f"[STEP] Step {self.current_step}: Taking action.")
        self._take_action(action)
        
        # Retrieve bus voltages
        all_bus_voltages = self.controller.circuit_all_bus_vmag_pu()
        if isinstance(all_bus_voltages, list):
            pass  # Already a list of floats
        elif isinstance(all_bus_voltages, str):
            all_bus_voltages = [float(v) for v in all_bus_voltages.split()]
        else:
            raise TypeError(f"Unexpected type for bus voltages: {type(all_bus_voltages)}")
        
        # Verify observation length matches expected
        expected_length = self.observation_space.shape[0]
        if len(all_bus_voltages) != expected_length:
            logger.error(f"[ERROR] Observation length {len(all_bus_voltages)} does not match expected {expected_length}.")
            raise ValueError(f"Observation length {len(all_bus_voltages)} does not match expected {expected_length}.")
        
        # Calculate voltage violations
        voltage_violations = sum(1 for v in all_bus_voltages if (v <= 0.95 or v >= 1.05) and v != 0)
        logger.debug(f"[OBSERVATION] Bus Voltages: {all_bus_voltages}")
        logger.info(f"[REWARD] Voltage Violations: {voltage_violations}")
        
        # Define reward
        reward = -voltage_violations
        
        # Update logs
        self.hourly_violations_count.append(voltage_violations)
        self.all_bus_voltages.append(all_bus_voltages)
        
        # Get observation
        observation = self.get_observation()
        
        # Determine if the episode is done
        termination = (self.control_steps >= self.max_control_steps) or (voltage_violations == 0)
        self.control_steps += 1
        
        if termination:
            logger.info(f"[TERMINATION] Episode terminated at step {self.control_steps} with reward {reward}.")
        
        # Return observation, reward, done, truncated, info
        return observation, reward, termination, False, {}
    
    def reset(self, *, seed=None, options=None):
        """
        Reset the environment to an initial state.
        """
        if seed is not None:
            np.random.seed(seed)
        if options is not None:
            logger.info(f"[RESET] Reset options: {options}")
        
        # Recompile OpenDSS to reset the state using the stored dss_file path
        self.controller.text(f"compile [{self.dss_file}]")
        logger.info(f"[RESET] Recompiled OpenDSS file: {self.dss_file}")
        
        # Update the current step
        self.current_step = (self.current_step + 1) % len(self.irradiance_profile)
        self.control_steps = 0
        
        # Apply load scales
        load_scales = np.random.normal(loc=self.load_profile[self.current_step], scale=0.001, size=len(self.load_names))
        logger.info(f"[RESET] Applying Load Scales for step {self.current_step}.")
        
        for load_name, load_scale in zip(self.load_names, load_scales):
            load_power = self.load_powers[self.load_names.index(load_name)]
            result = load_power * load_scale * self.load_scale
            self.controller.text(f"edit Load.{load_name} kW={result}")
            logger.debug(f"[LOAD] Edited Load.{load_name}: kW={result}")
        
        # Apply PV and Battery initial states based on irradiance
        irradiance = self.irradiance_profile[self.current_step]
        logger.info(f"[RESET] Current Irradiance: {irradiance}")
        
        for i in range(min(PV_KW_ACTION_LEN, len(self.ranked_buses))):
            bus = self.ranked_buses[i]
            fixed_power_factor = 0.9
            pv_kw = 100 * irradiance
            power_factor_angle = np.arccos(fixed_power_factor)
            pv_kvar = pv_kw * np.tan(power_factor_angle)
            battery_kw = self.load_profile[self.current_step] * 100 * self.load_scale
            self.controller.text(f"edit PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR={pv_kvar} Pmpp={pv_kw}")
            logger.debug(f"[LOAD] Edited PVSystem.PV{i+1} on Bus {bus}: kVAR={pv_kvar}, Pmpp={pv_kw}")
            
            self.controller.text(f"edit Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW={battery_kw} kVAR=0")
            logger.debug(f"[LOAD] Edited Storage.Battery{i+1} on Bus {bus}: kW={battery_kw}, kVAR=0")
        
        # Solve the power flow after resetting
        self.controller.text("set controlmode=off")
        self.controller.text("solve")
        logger.info(f"[RESET] Load and PV/Battery states updated and OpenDSS solved.")
        
        # Get initial observation
        observation = self.get_observation()
        
        # Return the initial observation
        return observation, {}
    
    def get_observation(self):
        """
        Retrieve the current observation (bus voltages).
        """
        all_bus_voltages = self.controller.circuit_all_bus_vmag_pu()
        if isinstance(all_bus_voltages, list):
            pass  # Already a list of floats
        elif isinstance(all_bus_voltages, str):
            all_bus_voltages = [float(v) for v in all_bus_voltages.split()]
        else:
            raise TypeError(f"Unexpected type for bus voltages: {type(all_bus_voltages)}")
        
        observation = np.array(all_bus_voltages).astype(np.float32).flatten()
        
        # Clip values to ensure they are within the observation space bounds
        observation = np.clip(observation, self.observation_space.low, self.observation_space.high)
        
        logger.debug(f"[GET_OBSERVATION] Retrieved Bus Voltages: {observation}")
        return observation

# Define the paths to your data files
dss_path = r"C:\Program Files\OpenDSS"  # Update if different
dss_file = r"C:\Users\Alaa\Documents\OpenDSS Model\OpenDSS Model (08.13.2020)\Master.dss"
irradiance_csv_file = r"D:\Alaa_Selim\Irradiacne_Profile_Iowa_State.csv"
load_profile_file = r"D:\Alaa_Selim\LoadShape1.csv"
load_powers_file = r"C:\Users\Alaa\Documents\OpenDSS Model\OpenDSS Model (08.13.2020)\Loaddemand.csv"

def env_creator(env_config):
    """
    Environment creator for registering with RLlib.
    """
    return PowerSystemEnv(
        dss_path=dss_path,
        dss_file=dss_file,
        irradiance_csv_file=irradiance_csv_file,
        load_profile_file=load_profile_file,
        load_powers_file=load_powers_file,
        seed=SEED,
        load_scale=5  # Adjust based on your system requirements
    )

# Register the environment with RLlib
register_env("PowerSystemEnv", env_creator)


In [2]:
from ray.rllib.algorithms.impala import ImpalaConfig
from ray import air
from ray import tune
config = ImpalaConfig()
# Print out some default values.
print(config.vtrace)  
# Update the config object.
config = config.training(   
    lr=tune.grid_search([0.0001])
)
# Set the config object's env.
config = config.resources(num_gpus=0)  
config = config.rollouts(num_rollout_workers=3) 
config = config.environment(env="PowerSystemEnv")  
# Use to_dict() to get the old-style python config dict
# when running with tune.
tune.Tuner(  
    "IMPALA",
    run_config=air.RunConfig(stop={"episodes_total": 100000}),
    param_space=config.to_dict(),
).fit()

(pid=126716) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(Impala pid=126716) 2024-11-07 11:48:47,049	WARNING algorithm_config.py:656 -- Cannot create ImpalaConfig from given `config_dict`! Property __stdout_file__ not supported.


(RolloutWorker pid=53900) OpenDSS Started successfully! 
(RolloutWorker pid=53900) OpenDSS Version 9.5.1.1 (64-bit build); License Status: Open 
(RolloutWorker pid=53900) 
(RolloutWorker pid=53900) 


(RolloutWorker pid=53900) [INIT] Number of buses (436) does not match expected observations (915). Please verify.
(pid=69940) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future! [repeated 3x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(RolloutWorker pid=69940) [INIT] Number of buses (436) does not match expected observations (915). Please verify.
(RolloutWorker pid=123460) [INIT] Number of buses (436) does not match expected observations (915). Please verify.
(RolloutWorker pid=123460) 2024-11-07 11:49:07,228	WARNING env.py:162 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
(Impala pid=126716) 2024-11-07 11:49:07,563	WARNING deprecation.py:50 -- DeprecationWarning: `LearningRateS

KeyboardInterrupt: 

In [3]:
from ray.rllib.algorithms.sac import SACConfig
from ray import air
from ray import tune

# Initialize SAC configuration
config = SACConfig()

# Update the training config
config = config.training(
    lr=3e-4,                      # Learning rate (common starting point for SAC)
    gamma=0.99,                   # Discount factor for future rewards
    tau=0.005,                    # Target smoothing coefficient
    initial_alpha=0.2,            # Initial entropy regularization coefficient
    train_batch_size=256,         # Number of samples per training batch
    target_entropy=-6,            # Target entropy based on action space dimensionality (manual setting)
)

# Set resources and configure the rollout workers
config = config.resources(
    num_gpus=0,                   # Number of GPUs to allocate
    num_cpus_per_worker=1         # Number of CPUs per rollout worker
)

config = config.rollouts(
    num_rollout_workers=5,        # Number of parallel rollout workers
    rollout_fragment_length=200,  # Number of steps per rollout fragment
    batch_mode="truncate_episodes"  # How to batch rollouts
)

# Define environment configuration
config = config.environment(
    env="PowerSystemEnv",
    env_config={
        "seed": 42,
        "load_scale": 1.75
    }
)

# Convert to dictionary for use with Tune
tune_config = config.to_dict()
logging.info("SAC configuration set.")

# ----------------------------
# 9. Initialize and Run the Tuner with SAC
# ----------------------------
tuner = tune.Tuner(
    "SAC",
    run_config=air.RunConfig(
        stop={"training_iteration": 1000},  # Set a suitable stop criteria
        checkpoint_config=air.CheckpointConfig(
            checkpoint_frequency=10,           # Enable checkpointing every 10 iterations
            checkpoint_at_end=True             # Save a checkpoint at the end
        ),
    ),
    param_space=tune_config,
)

logging.info("Starting SAC training...")
# Start the training process
results = tuner.fit()

# ----------------------------
# 10. Post-Training Analysis (Optional)
# ----------------------------
# Retrieve the DataFrame
results_df = results.get_dataframe()

# Print available columns for inspection
print("Available columns in the results DataFrame:")
print(results_df.columns.tolist())

(pid=40292) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(pid=40292) 2024-11-03 21:16:55,731	WARNING deprecation.py:50 -- DeprecationWarning: `build_tf_policy` has been deprecated. This will raise an error in the future!
(pid=40292) 2024-11-03 21:16:55,735	WARNING deprecation.py:50 -- DeprecationWarning: `build_policy_class` has been deprecated. This will raise an error in the future!
(SAC pid=40292) 2024-11-03 21:16:55,745	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/simple_q/` has been deprecated. Use `rllib_contrib/simple_q/` instead. This will raise an error in the future!
(SAC pid=40292) 2024-11-03 21:16:55,745	WARNING algorithm_config.py:656 -- Cannot create SACConfig from given `config_dict`! Property __stdout_file__ not supported.


(RolloutWorker pid=14560) OpenDSS Started successfully! 
(RolloutWorker pid=14560) OpenDSS Version 9.5.1.1 (64-bit build); License Status: Open 
(RolloutWorker pid=14560) 
(RolloutWorker pid=14560) 


(RolloutWorker pid=14560) [INIT] Number of buses (436) does not match expected observations (915). Please verify.
(RolloutWorker pid=112880) 2024-11-03 21:16:59,259	WARNING env.py:162 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
(RolloutWorker pid=112880) 2024-11-03 21:16:59,428	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.torch_modelv2.TorchModelV2` has been deprecated. Use `ray.rllib.core.rl_module.rl_module.RLModule` instead. This will raise an error in the future!
(RolloutWorker pid=112880) 2024-11-03 21:16:59,430	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.fcnet.FullyConnectedNetwork` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=112880) 2024-11-03 21:16:59,452	WARNING deprecation.py:50 -- DeprecationWarning: `TorchPolicy` has been deprecated. This will raise an error in the future!
(RolloutWorker

(SAC pid=40292) Caught sync error: Sync process failed: [WinError 32] Failed copying 'C:/Users/Alaa/ray_results/SAC/SAC_PowerSystemEnv_db146_00000_0_2024-11-03_21-16-52/checkpoint_000020/.is_checkpoint' to '/Users/Alaa/ray_results/SAC/SAC_PowerSystemEnv_db146_00000_0_2024-11-03_21-16-52/checkpoint_000020/.is_checkpoint'. Detail: [Windows error 32] The process cannot access the file because it is being used by another process.
(SAC pid=40292) . Retrying after sleeping for 1.0 seconds...
(SAC pid=40292) Caught sync error: Sync process failed: [WinError 32] Failed copying 'C:/Users/Alaa/ray_results/SAC/SAC_PowerSystemEnv_db146_00000_0_2024-11-03_21-16-52/checkpoint_000020/.is_checkpoint' to '/Users/Alaa/ray_results/SAC/SAC_PowerSystemEnv_db146_00000_0_2024-11-03_21-16-52/checkpoint_000020/.is_checkpoint'. Detail: [Windows error 32] The process cannot access the file because it is being used by another process.
(SAC pid=40292) . Retrying after sleeping for 1.0 seconds...
(SAC pid=40292) Co

(SAC pid=40292) Could not upload checkpoint to c://\Users\Alaa\ray_results\SAC\SAC_PowerSystemEnv_db146_00000_0_2024-11-03_21-16-52\checkpoint_000050 even after 3 retries.Please check if the credentials expired and that the remote filesystem is supported. For large checkpoints or artifacts, consider increasing `SyncConfig(sync_timeout)` (current value: 1800 seconds).
(SAC pid=40292) Last sync command failed: Sync process failed: [WinError 32] Failed copying 'C:/Users/Alaa/ray_results/SAC/SAC_PowerSystemEnv_db146_00000_0_2024-11-03_21-16-52/checkpoint_000050/.is_checkpoint' to '/Users/Alaa/ray_results/SAC/SAC_PowerSystemEnv_db146_00000_0_2024-11-03_21-16-52/checkpoint_000050/.is_checkpoint'. Detail: [Windows error 32] The process cannot access the file because it is being used by another process.
(SAC pid=40292) 
(SAC pid=40292) Caught sync error: Sync process failed: [WinError 32] Failed copying 'C:/Users/Alaa/ray_results/SAC/SAC_PowerSystemEnv_db146_00000_0_2024-11-03_21-16-52/checkpo

(SAC pid=40292) Caught sync error: Sync process failed: [WinError 32] Failed copying 'C:/Users/Alaa/ray_results/SAC/SAC_PowerSystemEnv_db146_00000_0_2024-11-03_21-16-52/checkpoint_000090/.is_checkpoint' to '/Users/Alaa/ray_results/SAC/SAC_PowerSystemEnv_db146_00000_0_2024-11-03_21-16-52/checkpoint_000090/.is_checkpoint'. Detail: [Windows error 32] The process cannot access the file because it is being used by another process.
(SAC pid=40292) . Retrying after sleeping for 1.0 seconds...
(SAC pid=40292) Caught sync error: Sync process failed: [WinError 32] Failed copying 'C:/Users/Alaa/ray_results/SAC/SAC_PowerSystemEnv_db146_00000_0_2024-11-03_21-16-52/checkpoint_000090/.is_checkpoint' to '/Users/Alaa/ray_results/SAC/SAC_PowerSystemEnv_db146_00000_0_2024-11-03_21-16-52/checkpoint_000090/.is_checkpoint'. Detail: [Windows error 32] The process cannot access the file because it is being used by another process.
(SAC pid=40292) . Retrying after sleeping for 1.0 seconds...
(SAC pid=40292) Co

(SAC pid=40292) Could not upload checkpoint to c://\Users\Alaa\ray_results\SAC\SAC_PowerSystemEnv_db146_00000_0_2024-11-03_21-16-52\checkpoint_000120 even after 3 retries.Please check if the credentials expired and that the remote filesystem is supported. For large checkpoints or artifacts, consider increasing `SyncConfig(sync_timeout)` (current value: 1800 seconds).
(SAC pid=40292) Last sync command failed: Sync process failed: [WinError 32] Failed copying 'C:/Users/Alaa/ray_results/SAC/SAC_PowerSystemEnv_db146_00000_0_2024-11-03_21-16-52/checkpoint_000120/.is_checkpoint' to '/Users/Alaa/ray_results/SAC/SAC_PowerSystemEnv_db146_00000_0_2024-11-03_21-16-52/checkpoint_000120/.is_checkpoint'. Detail: [Windows error 32] The process cannot access the file because it is being used by another process.
(SAC pid=40292) 
(SAC pid=40292) Caught sync error: Sync process failed: [WinError 32] Failed copying 'C:/Users/Alaa/ray_results/SAC/SAC_PowerSystemEnv_db146_00000_0_2024-11-03_21-16-52/checkpo

(SAC pid=40292) Caught sync error: Sync process failed: [WinError 32] Failed copying 'C:/Users/Alaa/ray_results/SAC/SAC_PowerSystemEnv_db146_00000_0_2024-11-03_21-16-52/checkpoint_000160/.is_checkpoint' to '/Users/Alaa/ray_results/SAC/SAC_PowerSystemEnv_db146_00000_0_2024-11-03_21-16-52/checkpoint_000160/.is_checkpoint'. Detail: [Windows error 32] The process cannot access the file because it is being used by another process.
(SAC pid=40292) . Retrying after sleeping for 1.0 seconds...
(SAC pid=40292) Caught sync error: Sync process failed: [WinError 32] Failed copying 'C:/Users/Alaa/ray_results/SAC/SAC_PowerSystemEnv_db146_00000_0_2024-11-03_21-16-52/checkpoint_000160/.is_checkpoint' to '/Users/Alaa/ray_results/SAC/SAC_PowerSystemEnv_db146_00000_0_2024-11-03_21-16-52/checkpoint_000160/.is_checkpoint'. Detail: [Windows error 32] The process cannot access the file because it is being used by another process.
(SAC pid=40292) . Retrying after sleeping for 1.0 seconds...
(SAC pid=40292) Co

(SAC pid=40292) Could not upload checkpoint to c://\Users\Alaa\ray_results\SAC\SAC_PowerSystemEnv_db146_00000_0_2024-11-03_21-16-52\checkpoint_000190 even after 3 retries.Please check if the credentials expired and that the remote filesystem is supported. For large checkpoints or artifacts, consider increasing `SyncConfig(sync_timeout)` (current value: 1800 seconds).
(SAC pid=40292) Last sync command failed: Sync process failed: [WinError 32] Failed copying 'C:/Users/Alaa/ray_results/SAC/SAC_PowerSystemEnv_db146_00000_0_2024-11-03_21-16-52/checkpoint_000190/.is_checkpoint' to '/Users/Alaa/ray_results/SAC/SAC_PowerSystemEnv_db146_00000_0_2024-11-03_21-16-52/checkpoint_000190/.is_checkpoint'. Detail: [Windows error 32] The process cannot access the file because it is being used by another process.
(SAC pid=40292) 
(SAC pid=40292) Caught sync error: Sync process failed: [WinError 32] Failed copying 'C:/Users/Alaa/ray_results/SAC/SAC_PowerSystemEnv_db146_00000_0_2024-11-03_21-16-52/checkpo

KeyboardInterrupt: 

In [3]:
import os
from ray.rllib.algorithms.sac import SACConfig
from ray import air
from ray import tune

# Optional: Set a different ray_results directory to avoid path issues
ray_results_dir = r"D:\ray_results\SAC"
os.makedirs(ray_results_dir, exist_ok=True)

# Initialize SAC configuration
config = SACConfig()

# Update the config object with training parameters
config = config.training(   
    lr=tune.grid_search([0.0001])  # Expand this grid search as needed
)

# Define resource allocations
config = config.resources(num_gpus=0)  

# Define rollout workers based on your system's CPU cores
config = config.rollouts(num_rollout_workers=5)  # Start with 1 to reduce concurrency

# Set the environment and specify max_episode_steps
config = config.environment(env="PowerSystemEnv", env_config={"max_episode_steps": 1000})

# Specify the results directory
config = config.debugging(log_level="INFO")  # Set to DEBUG for more detailed logs if needed

# Initialize the tuner with the updated config
tuner = tune.Tuner(  
    "SAC",  # Ensure the algorithm name matches
    run_config=air.RunConfig(
        stop={"episodes_total": 10000},
        local_dir=ray_results_dir,  # Direct results to the specified directory
    ),
    param_space=config.to_dict(),
)

# Start the training process
results = tuner.fit()


(pid=87756) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(pid=87756) 2024-11-03 23:53:35,315	WARNING deprecation.py:50 -- DeprecationWarning: `build_tf_policy` has been deprecated. This will raise an error in the future!
(pid=87756) 2024-11-03 23:53:35,319	WARNING deprecation.py:50 -- DeprecationWarning: `build_policy_class` has been deprecated. This will raise an error in the future!
(SAC pid=87756) 2024-11-03 23:53:35,330	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/simple_q/` has been deprecated. Use `rllib_contrib/simple_q/` instead. This will raise an error in the future!
(SAC pid=87756) 2024-11-03 23:53:35,332	WARNING algorithm_config.py:656 -- Cannot create SACConfig from given `config_dict`! Property __stdout_file__ not supported.
(RolloutWorker pid=111132) [INIT] Number of buses (436) does not match expected observations (915). Please verify.
(RolloutWorker pid=66240) 2024-11-03 23:53:39,211	WARNIN

(RolloutWorker pid=111132) OpenDSS Started successfully! 
(RolloutWorker pid=111132) OpenDSS Version 9.5.1.1 (64-bit build); License Status: Open 
(RolloutWorker pid=111132) 
(RolloutWorker pid=111132) 


(RolloutWorker pid=66240) 2024-11-03 23:53:39,407	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.torch_modelv2.TorchModelV2` has been deprecated. Use `ray.rllib.core.rl_module.rl_module.RLModule` instead. This will raise an error in the future!
(RolloutWorker pid=66240) 2024-11-03 23:53:39,409	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.fcnet.FullyConnectedNetwork` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=66240) 2024-11-03 23:53:39,430	WARNING deprecation.py:50 -- DeprecationWarning: `TorchPolicy` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=66240) 2024-11-03 23:53:39,431	WARNING deprecation.py:50 -- DeprecationWarning: `StochasticSampling` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=66240) 2024-11-03 23:53:39,431	WARNING deprecation.py:50 -- DeprecationWarning: `Exploration` has been deprecated. This will raise an error in

(SAC pid=87756) 2024-11-03 23:53:49,950	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.execution.train_ops.multi_gpu_train_one_step` has been deprecated. This will raise an error in the future!
(pid=26736) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future! [repeated 5x across cluster]
(RolloutWorker pid=26736) 2024-11-03 23:53:39,158	WARNING deprecation.py:50 -- DeprecationWarning: `build_tf_policy` has been deprecated. This will raise an error in the future! [repeated 5x across cluster]
(RolloutWorker pid=26736) 2024-11-03 23:53:39,162	WARNING deprecation.py:50 -- DeprecationWarning: `build_policy_class` has been deprecated. This will raise an error in the future! [repeated 5x across cluster]
(RolloutWorker pid=26736) [INIT] Number of buses (436) does not match expected observations (915). Please verify. [repeated 4x across cluster]
(RolloutWorker pid=26736) 2024-11-03 23:53:39,433	INFO policy.py:1294 -- Policy (worker=2

In [4]:
import os
from ray.rllib.algorithms.ppo import PPOConfig  # Changed from SACConfig to PPOConfig
from ray import air
from ray import tune

# Optional: Set a different ray_results directory to avoid path issues
ray_results_dir = r"D:\ray_results\PPO"  # Updated directory name to PPO
os.makedirs(ray_results_dir, exist_ok=True)

# Initialize PPO configuration
config = PPOConfig()  # Changed from SACConfig() to PPOConfig()

# Update the config object with training parameters
config = config.training(
    lr=tune.grid_search([0.0001])  # Expand this grid search as needed
)

# Define resource allocations
config = config.resources(num_gpus=0)  

# Define rollout workers based on your system's CPU cores
config = config.rollouts(num_rollout_workers=5)  # Start with 1 to reduce concurrency

# Set the environment and specify max_episode_steps
config = config.environment(env="PowerSystemEnv", env_config={"max_episode_steps": 1000})

# Specify the results directory
config = config.debugging(log_level="INFO")  # Set to DEBUG for more detailed logs if needed

# Initialize the tuner with the updated config
tuner = tune.Tuner(
    "PPO",  # Changed algorithm name from "SAC" to "PPO"
    run_config=air.RunConfig(
        stop={"episodes_total": 10000},
        local_dir=ray_results_dir,  # Direct results to the specified directory
    ),
    param_space=config.to_dict(),
)

# Start the training process
results = tuner.fit()


2024-11-04 01:07:27,957	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
(pid=1932) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(PPO pid=1932) 2024-11-04 01:07:31,126	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}

(RolloutWorker pid=107548) OpenDSS Started successfully! 
(RolloutWorker pid=107548) OpenDSS Version 9.5.1.1 (64-bit build); License Status: Open 
(RolloutWorker pid=107548) 
(RolloutWorker pid=107548) 


(RolloutWorker pid=68320) [INIT] Number of buses (436) does not match expected observations (915). Please verify.
(RolloutWorker pid=107548) 2024-11-04 01:07:34,445	WARNING env.py:162 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
(RolloutWorker pid=107548) 2024-11-04 01:07:34,507	INFO policy.py:1294 -- Policy (worker=1) running on CPU.
(RolloutWorker pid=107548) 2024-11-04 01:07:34,540	INFO torch_policy_v2.py:113 -- Found 1 visible cuda devices.
(RolloutWorker pid=107548) 2024-11-04 01:07:34,541	WARNING deprecation.py:50 -- DeprecationWarning: `ValueNetworkMixin` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=107548) 2024-11-04 01:07:34,541	WARNING deprecation.py:50 -- DeprecationWarning: `LearningRateSchedule` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=107548) 2024-11-04 01:07:34,541	WARNING deprecation.py:50 -- Deprecatio

2024-11-04 01:12:09,545	WARNING tune.py:1122 -- Trial Runner checkpointing failed: Sync process failed: GetFileInfo() yielded path 'C:/Users/Alaa/ray_results/PPO', which is outside base dir 'C:\Users\Alaa\ray_results\PPO'
2024-11-04 01:12:10,059	INFO tune.py:1148 -- Total run time: 282.12 seconds (281.57 seconds for the tuning loop).


In [3]:
import os
from ray.rllib.algorithms.impala import ImpalaConfig
from ray import air
from ray import tune

# Optional: Set a different ray_results directory to avoid path issues
ray_results_dir = r"D:\ray_results\IMPALA"
os.makedirs(ray_results_dir, exist_ok=True)

# Initialize IMPALA configuration
config = ImpalaConfig()

# Print out some default values for debugging
print("V-trace:", config.vtrace)  

# Update the config object with training parameters
config = config.training(   
    lr=tune.grid_search([0.0001])  # Expand this grid search as needed
)

# Define resource allocations
config = config.resources(num_gpus=1)  

# Define rollout workers based on your system's CPU cores
config = config.rollouts(num_rollout_workers=3)  # Start with 1 to reduce concurrency

# Set the environment and specify max_episode_steps
config = config.environment(env="PowerSystemEnv", env_config={"max_episode_steps": 1000})

# Optionally, define a maximum episode length if not set in the environment
# (Redundant if already set in environment metadata)

# Set the log level for debugging
config = config.debugging(log_level="INFO")  # Set to DEBUG for more detailed logs if needed

# Initialize the tuner with the updated config
tuner = tune.Tuner(  
    "IMPALA",  # Ensure the algorithm name matches
    run_config=air.RunConfig(
        stop={"episodes_total": 10000},
        local_dir=ray_results_dir,  # Direct results to the specified directory
    ),
    param_space=config.to_dict(),
)

# Start the training process
results = tuner.fit()


(pid=15172) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(Impala pid=15172) 2024-11-03 23:44:35,467	WARNING algorithm_config.py:656 -- Cannot create ImpalaConfig from given `config_dict`! Property __stdout_file__ not supported.


(RolloutWorker pid=60388) OpenDSS Started successfully! 
(RolloutWorker pid=60388) OpenDSS Version 9.5.1.1 (64-bit build); License Status: Open 
(RolloutWorker pid=60388) 
(RolloutWorker pid=60388) 


(RolloutWorker pid=60388) [INIT] Number of buses (436) does not match expected observations (915). Please verify.
(RolloutWorker pid=60388) 2024-11-03 23:44:38,887	WARNING env.py:162 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
(RolloutWorker pid=60388) 2024-11-03 23:44:39,001	WARNING deprecation.py:50 -- DeprecationWarning: `LearningRateSchedule` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=60388) 2024-11-03 23:44:39,002	WARNING deprecation.py:50 -- DeprecationWarning: `EntropyCoeffSchedule` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=60388) 2024-11-03 23:44:39,004	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.fcnet.FullyConnectedNetwork` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=60388) 2024-11-03 23:44:39,004	WARNING deprecation.py:50 -- DeprecationWarning

(Impala pid=15172) 2024-11-03 23:44:42,312	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.execution.replay_ops.SimpleReplayBuffer` has been deprecated. This will raise an error in the future!
(Impala pid=15172) 2024-11-03 23:44:42,341	INFO replay_buffer.py:63 -- Estimated max memory usage for replay buffer is 0.0004241 GB (1 batches of size 50, 424100 bytes each), available system memory is 137.221013504 GB
(RolloutWorker pid=60388) 2024-11-03 23:44:42,310	INFO rollout_worker.py:732 -- Completed sample batch:
(RolloutWorker pid=60388) { 'count': 50,
(RolloutWorker pid=60388)   'policy_batches': { 'default_policy': { 'action_dist_inputs': np.ndarray((50, 186), dtype=float32, min=-0.015, max=0.016, mean=-0.001),
(RolloutWorker pid=60388)                                           'action_logp': np.ndarray((50,), dtype=float32, min=-145.739, max=-119.272, mean=-131.716),
(RolloutWorker pid=60388)                                           'actions': np.ndarray((50, 93), dtype=f

In [3]:
import os
from ray.rllib.algorithms.impala import ImpalaConfig
from ray import air
from ray import tune

# Optional: Set a different ray_results directory to avoid path issues
ray_results_dir = r"D:\ray_results\IMPALA"
os.makedirs(ray_results_dir, exist_ok=True)

# Initialize IMPALA configuration
config = ImpalaConfig()

# Print out some default values for debugging
print("V-trace:", config.vtrace)  

# Update the config object with training parameters
config = config.training(   
    lr=tune.grid_search([0.0001])  # Expand this grid search as needed
)

# Define resource allocations
config = config.resources(num_gpus=1)  

# Define rollout workers based on your system's CPU cores
config = config.rollouts(num_rollout_workers=3)  # Start with 1 to reduce concurrency

# Set the environment and specify max_episode_steps
config = config.environment(env="PowerSystemEnv", env_config={"max_episode_steps": 1000})

# Optionally, define a maximum episode length if not set in the environment
# (Redundant if already set in environment metadata)

# Set the log level for debugging
config = config.debugging(log_level="INFO")  # Set to DEBUG for more detailed logs if needed

# Initialize the tuner with the updated config
tuner = tune.Tuner(  
    "IMPALA",  # Ensure the algorithm name matches
    run_config=air.RunConfig(
        stop={"episodes_total": 10000},
        local_dir=ray_results_dir,  # Direct results to the specified directory
    ),
    param_space=config.to_dict(),
)

# Start the training process
results = tuner.fit()


(pid=72824) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(Impala pid=72824) 2024-11-07 13:42:35,883	WARNING algorithm_config.py:656 -- Cannot create ImpalaConfig from given `config_dict`! Property __stdout_file__ not supported.
(RolloutWorker pid=58728) [INIT] Number of buses (436) does not match expected observations (915). Please verify.
(RolloutWorker pid=58728) 2024-11-07 13:42:40,094	WARNING env.py:162 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
(Impala pid=72824) 2024-11-07 13:42:40,244	INFO worker_set.py:297 -- Inferred observation/action spaces from remote worker (local worker has no env): {'default_policy': (Box(0.0, 2.0, (915,), float32), Box([-800.  -800.  -800.  -800.  -800.  -800.  -800.  -800.  -800.  -800.
(Impala pid=72824)  -800.  -800.  -800.  -800.  -800.  -800.  -800.  -800.  -800.  -800.
(Impala pid=72824)  -800.

(RolloutWorker pid=58728) OpenDSS Started successfully! 
(RolloutWorker pid=58728) OpenDSS Version 9.5.1.1 (64-bit build); License Status: Open 
(RolloutWorker pid=58728) 
(RolloutWorker pid=58728) 


(pid=84360) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future! [repeated 3x across cluster]
(Impala pid=72824) 2024-11-07 13:42:41,911	INFO rollout_worker.py:1742 -- Built policy map: <PolicyMap lru-caching-capacity=100 policy-IDs=['default_policy']>
(Impala pid=72824) 2024-11-07 13:42:41,911	INFO rollout_worker.py:1743 -- Built preprocessor map: {'default_policy': None}
(Impala pid=72824) 2024-11-07 13:42:41,911	INFO rollout_worker.py:550 -- Built filter map: defaultdict(<class 'ray.rllib.utils.filter.NoFilter'>, {})
(Impala pid=72824) 2024-11-07 13:42:41,917	INFO impala.py:513 -- Enabling multi-GPU mode, 1 GPUs, 1 parallel tower-stacks
(Impala pid=72824) 2024-11-07 13:42:41,917	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.execution.multi_gpu_learner_thread.MultiGPULearnerThread` has been deprecated. This will raise an error in the future!
(Impala pid=72824) 2024-11-07 13:42:41,917	WARNING deprecation.py:50 -- Deprecat

In [2]:
import os
from ray.rllib.algorithms.impala import ImpalaConfig
from ray import air
from ray import tune

# ------------------------------
# 1. Setup Ray Results Directory
# ------------------------------

# Define a clean Ray results directory to avoid path issues
ray_results_dir = r"D:\ray_results\IMPALA"
os.makedirs(ray_results_dir, exist_ok=True)

# ------------------------------
# 2. Initialize IMPALA Configuration
# ------------------------------

# Initialize the IMPALA configuration
config = ImpalaConfig()

# ------------------------------
# 3. Update Training Parameters
# ------------------------------

# Update training parameters to enhance stability
config = config.training(
    lr=1e-4,                    # Learning rate set to a standard value
    grad_clip=40.0,             # Gradient clipping to prevent exploding gradients
    vf_loss_coeff=0.5,          # Value function loss coefficient
    entropy_coeff=0.01,          # Entropy coefficient for exploration
    # You can adjust these hyperparameters based on your specific needs
)

# ------------------------------
# 4. Configure Resources
# ------------------------------

config = config.resources(
    num_gpus=1,                 # No GPU usage; set to >0 if GPUs are available
    num_cpus_per_worker=1,      # Allocate one CPU per worker
)

# ------------------------------
# 5. Configure Rollouts
# ------------------------------

config = config.rollouts(
    num_rollout_workers=3,      # Start with 1 worker to reduce concurrency issues
    rollout_fragment_length=200 # Length of each rollout fragment
       # Size of the training batch
)

# ------------------------------
# 6. Set the Environment
# ------------------------------

config = config.environment(
    env="PowerSystemEnv",        # Your custom environment
    env_config={
        "max_episode_steps": 1000  # Define maximum steps per episode
    }
)

# ------------------------------
# 7. Configure the Model
# ------------------------------



# ------------------------------
# 8. Enable Checkpointing
# ------------------------------


# ------------------------------
# 9. Set Debugging and Logging
# ------------------------------

config = config.debugging(
    log_level="INFO"              # Set to "DEBUG" for more detailed logs if needed
)

# ------------------------------
# 10. Initialize the Tuner
# ------------------------------

# Initialize the tuner with the updated configuration
tuner = tune.Tuner(
    "IMPALA",                      # Specify the IMPALA algorithm
    run_config=air.RunConfig(
        stop={"episodes_total": 100000},    # Define stopping criteria
        local_dir=ray_results_dir,          # Set the Ray results directory
        verbose=1,                           # Set verbosity level (0=quiet, 1=normal, 2=debug)
    ),
    param_space=config.to_dict(),              # Pass the IMPALA configuration
)

# ------------------------------
# 11. Start the Training Process
# ------------------------------

# Start the training process
results = tuner.fit()


(pid=11636) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(Impala pid=11636) 2024-11-03 22:13:56,342	WARNING algorithm_config.py:656 -- Cannot create ImpalaConfig from given `config_dict`! Property __stdout_file__ not supported.
(RolloutWorker pid=13196) [INIT] Number of buses (436) does not match expected observations (915). Please verify.
(RolloutWorker pid=69308) 2024-11-03 22:13:59,707	WARNING env.py:162 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
(RolloutWorker pid=13196) 2024-11-03 22:13:59,800	INFO policy.py:1294 -- Policy (worker=2) running on CPU.
(RolloutWorker pid=13196) 2024-11-03 22:13:59,800	INFO torch_policy_v2.py:113 -- Found 1 visible cuda devices.
(RolloutWorker pid=69308) 2024-11-03 22:13:59,793	WARNING deprecation.py:50 -- DeprecationWarning: `LearningRateSchedule` has been deprecated. This will raise an error in t

2024-11-03 22:13:59,900	WARNING tune.py:1122 -- Trial Runner checkpointing failed: Sync process failed: GetFileInfo() yielded path 'C:/Users/Alaa/ray_results/IMPALA', which is outside base dir 'C:\Users\Alaa\ray_results\IMPALA'


(RolloutWorker pid=13196) OpenDSS Started successfully! 
(RolloutWorker pid=13196) OpenDSS Version 9.5.1.1 (64-bit build); License Status: Open 
(RolloutWorker pid=13196) 
(RolloutWorker pid=13196) 


2024-11-03 22:13:59,904	ERROR tune.py:1144 -- Trials did not complete: [IMPALA_PowerSystemEnv_d1ea4_00000]
2024-11-03 22:13:59,905	INFO tune.py:1148 -- Total run time: 6.78 seconds (6.73 seconds for the tuning loop).
2024-11-03 22:13:59,921	WARNING experiment_analysis.py:916 -- Failed to read the results for 1 trials:
- C:\Users\Alaa\ray_results\IMPALA\IMPALA_PowerSystemEnv_d1ea4_00000_0_2024-11-03_22-13-53
(Impala pid=11636) 2024-11-03 22:13:59,875	ERROR actor_manager.py:500 -- Ray error, taking actor 1 out of service. The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=69308, ip=127.0.0.1, actor_id=d81452b2140248c8308fed8901000000, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x0000023F6BCE7A00>)
(Impala pid=11636)   File "python\ray\_raylet.pyx", line 1424, in ray._raylet.execute_task
(Impala pid=11636)   File "python\ray\_raylet.pyx", line 1364, in ray._raylet.execute_task.function_executor
(Impala pid=11636)   Fil

In [26]:
# Import required modules
import numpy as np
from ray.tune.registry import register_env
import logging


# Configure logging to see outputs directly in the notebook
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Define paths to DSS file and data files (adjust paths accordingly)
dss_path = r"C:\Program Files\OpenDSS"  # Adjust path if different
dss_file = r"C:\Users\Alaa\Documents\OpenDSS Model\OpenDSS Model (08.13.2020)\Master.dss"
irradiance_csv_file = r"D:\Alaa_Selim\Irradiance_Profile_Santa_Clara.csv"
load_profile_file = r"D:\Alaa_Selim\LoadShape1.csv"
load_powers_file = r"C:\Users\Alaa\Documents\OpenDSS Model\OpenDSS Model (08.13.2020)\Loaddemand.csv"

# Initialize the environment directly
env = PowerSystemEnv(
    dss_path=dss_path,
    dss_file=dss_file,
    irradiance_csv_file=irradiance_csv_file,
    load_profile_file=load_profile_file,
    load_powers_file=load_powers_file,
    seed=42,
    load_scale=1.75
)

# Reset environment and get initial observation
observation, _ = env.reset()

# Print the number of observations in the observation array
num_observations = len(observation)
print(f"Number of observations: {num_observations}")


OpenDSS Started successfully! 
OpenDSS Version 9.5.1.1 (64-bit build); License Status: Open 


Number of observations: 915


In [ ]:
import os
import ray
from ray.rllib.algorithms.impala import Impala
from ray.tune.registry import register_env
import gymnasium as gym
from gymnasium import spaces
import numpy as np
from py_dss_interface import DSSDLL
import pandas as pd
import torch
import warnings
import logging
import csv

# ------------------------------
# 1. Environment Definition
# ------------------------------

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Configure logging
logging.basicConfig(level=logging.INFO)  # Set to DEBUG for more detailed logs
logger = logging.getLogger(__name__)

# Define lengths for each segment of the action space
PV_KVAR_ACTION_LEN = 30          # Number of PV systems for reactive power control
PV_KW_ACTION_LEN = 30            # Number of PV systems for active power control
BESS_KW_ACTION_LEN = 30          # Number of Batteries for active power control
TRANSFORMER_TAPS_ACTION_LEN = 1  # Number of transformer taps
CAPACITOR_ACTION_LEN = 2         # Number of Capacitors (Manually defined)

# Define action value ranges for each controllable element
PV_KVAR_ACTION_LOW = -800
PV_KVAR_ACTION_HIGH = 800
PV_KW_ACTION_LOW = 0
PV_KW_ACTION_HIGH = 1000
BESS_KW_ACTION_LOW = 0
BESS_KW_ACTION_HIGH = 1000
TRANSFORMER_TAPS_ACTION_LOW = 0.9
TRANSFORMER_TAPS_ACTION_HIGH = 1.1
CAPACITOR_ACTION_LOW = 0
CAPACITOR_ACTION_HIGH = 1

# Set a seed for reproducibility
SEED = 42

class PowerSystemEnv(gym.Env):
    """
    Custom Environment for Power System Control using OpenDSS and Gymnasium.
    """
    metadata = {'render.modes': ['human'], 'spec.max_episode_steps': 1000}

    def __init__(self, dss_path, dss_file, irradiance_csv_file, load_profile_file, load_powers_file, seed=None, load_scale=3):
        super(PowerSystemEnv, self).__init__()
        self.seed(seed)
        
        # Store the OpenDSS file path for later use
        self.dss_file = dss_file
        
        # Initialize OpenDSS Controller
        self.controller = DSSDLL(dss_path)
        self.controller.text(f"compile [{dss_file}]")
        logger.info(f"[INIT] Compiled OpenDSS file: {dss_file}")
        
        # Manually define capacitor names
        self.capacitor_names = ["CAP_201", "CAP_301"]
        logger.info(f"[INIT] Defined capacitors: {self.capacitor_names}")
        
        # Retrieve bus names from OpenDSS
        self.ranked_buses = self.get_bus_names()
        logger.info(f"[INIT] Retrieved {len(self.ranked_buses)} buses from OpenDSS.")
        
        # Manually define load names
        self.load_names = [
            "Load_1003", "Load_1004", "Load_1005", "Load_1006", "Load_1007", "Load_1008",
            "Load_1009", "Load_1010", "Load_1011", "Load_1012", "Load_1013", "Load_1014",
            "Load_1015", "Load_1016", "Load_1017", "Load_2002", "Load_2003", "Load_2005",
            "Load_2008", "Load_2009", "Load_2010", "Load_2011", "Load_2014", "Load_2015",
            "Load_2016", "Load_2017", "Load_2018", "Load_2020", "Load_2022", "Load_2023",
            "Load_2024", "Load_2025", "Load_2028", "Load_2029", "Load_2030", "Load_2031",
            "Load_2032", "Load_2034", "Load_2035", "Load_2037", "Load_2040", "Load_2041",
            "Load_2042", "Load_2043", "Load_2045", "Load_2046", "Load_2047", "Load_2048",
            "Load_2049", "Load_2050", "Load_2051", "Load_2052", "Load_2053", "Load_2054",
            "Load_2055", "Load_2056", "Load_2058", "Load_2059", "Load_2060",
            "Load_3002", "Load_3004", "Load_3006", "Load_3007", "Load_3009", "Load_3010",
            "Load_3011", "Load_3012", "Load_3013", "Load_3014", "Load_3016", "Load_3017",
            "Load_3018", "Load_3019", "Load_3020", "Load_3021", "Load_3023", "Load_3024",
            "Load_3025", "Load_3026", "Load_3027", "Load_3028", "Load_3029", "Load_3031",
            "Load_3032", "Load_3033", "Load_3034", "Load_3035", "Load_3036", "Load_3037",
            "Load_3038", "Load_3039", "Load_3041", "Load_3042", "Load_3043", "Load_3044",
            "Load_3045", "Load_3047", "Load_3048", "Load_3049", "Load_3050", "Load_3051",
            "Load_3052", "Load_3054", "Load_3056", "Load_3057", "Load_3058", "Load_3059",
            "Load_3060", "Load_3061", "Load_3062", "Load_3063", "Load_3064", "Load_3065",
            "Load_3066", "Load_3067", "Load_3070", "Load_3071", "Load_3072", "Load_3073",
            "Load_3074", "Load_3077", "Load_3078", "Load_3081", "Load_3083", "Load_3084",
            "Load_3085", "Load_3086", "Load_3087", "Load_3088", "Load_3089", "Load_3090",
            "Load_3091", "Load_3093", "Load_3094", "Load_3095", "Load_3096", "Load_3097",
            "Load_3098", "Load_3099", "Load_3101", "Load_3102", "Load_3103", "Load_3104",
            "Load_3105", "Load_3106", "Load_3108", "Load_3109", "Load_3110", "Load_3111",
            "Load_3112", "Load_3114", "Load_3115", "Load_3116", "Load_3117", "Load_3120",
            "Load_3121", "Load_3122", "Load_3123", "Load_3124", "Load_3125", "Load_3126",
            "Load_3127", "Load_3128", "Load_3129", "Load_3130", "Load_3131", "Load_3132",
            "Load_3134", "Load_3135", "Load_3136", "Load_3137", "Load_3138", "Load_3139",
            "Load_3141", "Load_3142", "Load_3143", "Load_3144", "Load_3145", "Load_3146",
            "Load_3147", "Load_3148", "Load_3149", "Load_3150", "Load_3151", "Load_3152",
            "Load_3153", "Load_3154", "Load_3155", "Load_3157", "Load_3158", "Load_3159",
            "Load_3160", "Load_3161", "Load_3162"
        ]
        logger.info(f"[INIT] Defined loads: {len(self.load_names)} loads loaded.")
        
        # Validate load names against OpenDSS
        self.validate_loads()
        
        # Define system parameters
        self.KWrated = 1000
        self.load_scale = load_scale
        
        # Initialize PV systems and Batteries
        num_pv = PV_KW_ACTION_LEN  # 30 PV systems
        num_battery = BESS_KW_ACTION_LEN  # 30 Batteries
        logger.info(f"[INIT] Initializing {num_pv} PV systems and {num_battery} Batteries.")
        
        for i in range(num_pv):
            if i < len(self.ranked_buses):
                bus = self.ranked_buses[i]
                self.controller.text(f"new PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR=0 KVA=100 Pmpp=80")
                logger.debug(f"[INIT] Initialized PVSystem.PV{i+1} on Bus {bus}")
            else:
                logger.warning(f"[INIT] Not enough buses to initialize PVSystem.PV{i+1}")
        
        for i in range(num_battery):
            if i < len(self.ranked_buses):
                bus = self.ranked_buses[i]
                self.controller.text(f"new Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW=100 kVAR=0")
                logger.debug(f"[INIT] Initialized Storage.Battery{i+1} on Bus {bus}")
            else:
                logger.warning(f"[INIT] Not enough buses to initialize Storage.Battery{i+1}")
        
        # Load profiles
        self.irradiance_profile = self.load_csv(irradiance_csv_file, "Irradiance")
        self.load_profile = self.load_csv(load_profile_file, "Load Profile", skip_header=True)
        self.load_powers = self.load_csv(load_powers_file, "Load Powers", skip_header=False)
        
        # Validate load powers
        if len(self.load_powers) != len(self.load_names):
            raise ValueError("Number of load powers does not match number of load names.")
        logger.info("[INIT] Load powers and load names are consistent.")
        
        # Define action and observation spaces
        self.action_space = spaces.Box(
            low=np.array(
                [PV_KVAR_ACTION_LOW] * PV_KVAR_ACTION_LEN +
                [PV_KW_ACTION_LOW] * PV_KW_ACTION_LEN +
                [BESS_KW_ACTION_LOW] * BESS_KW_ACTION_LEN +
                [TRANSFORMER_TAPS_ACTION_LOW] * TRANSFORMER_TAPS_ACTION_LEN +
                [CAPACITOR_ACTION_LOW] * CAPACITOR_ACTION_LEN
            ),
            high=np.array(
                [PV_KVAR_ACTION_HIGH] * PV_KVAR_ACTION_LEN +
                [PV_KW_ACTION_HIGH] * PV_KW_ACTION_LEN +
                [BESS_KW_ACTION_HIGH] * BESS_KW_ACTION_LEN +
                [TRANSFORMER_TAPS_ACTION_HIGH] * TRANSFORMER_TAPS_ACTION_LEN +
                [CAPACITOR_ACTION_HIGH] * CAPACITOR_ACTION_LEN
            ),
            dtype=np.float32
        )
        logger.info(f"[INIT] Defined action space with shape: {self.action_space.shape}")
        
        # Update the number of observations based on current setup
        num_buses = len(self.ranked_buses)
        expected_observations = 915
        if num_buses != expected_observations:
            logger.warning(f"[INIT] Number of buses ({num_buses}) does not match expected observations ({expected_observations}). Please verify.")
        
        self.observation_space = spaces.Box(
            low=0.0,
            high=2.0,
            shape=(expected_observations,),
            dtype=np.float32
        )
        logger.info(f"[INIT] Defined observation space with shape: {self.observation_space.shape}")
        
        # Initialize other variables
        self.current_step = 0
        self.control_steps = 0
        self.max_control_steps = 1000  # Define as needed
        self.hourly_violations_count = []
        self.all_bus_voltages = []
    
    def load_csv(self, file_path, description, skip_header=False):
        """
        Utility method to load CSV files.
        """
        try:
            with open(file_path, 'r') as csvfile:
                reader = csv.reader(csvfile)
                if skip_header:
                    next(reader, None)  # Skip header if present
                data = [float(row[0]) for row in reader]
            logger.info(f"[LOAD] Loaded {description} with {len(data)} entries.")
            return data
        except Exception as e:
            logger.error(f"[LOAD] Failed to load {description} from {file_path}: {e}")
            raise e
    
    def validate_loads(self):
        """
        Validates that all load names exist in the OpenDSS model.
        """
        # Since 'circuit_all_load_names()' is unavailable, manual validation is limited.
        # Ensure that the load names in self.load_names match those in OpenDSS.
        # Alternatively, proceed with caution.
        logger.info("[VALIDATE] Manual validation of load names is required.")
    
    def seed(self, seed=None):
        """
        Set the seed for reproducibility.
        """
        self.seed_value = seed
        np.random.seed(seed)
        torch.manual_seed(seed)
        if seed is not None:
            torch.cuda.manual_seed_all(seed)
        logger.info(f"[SEED] Environment seeded with value: {seed}")
    
    def get_bus_names(self):
        """
        Retrieve all bus names from OpenDSS.
        """
        bus_names = self.controller.circuit_all_bus_names()
        if isinstance(bus_names, list):
            logger.info(f"[INFO] Retrieved {len(bus_names)} bus names.")
            return bus_names
        else:
            raise TypeError(f"Expected list for bus names, got {type(bus_names)}")
    
    def _take_action(self, action):
        """
        Apply the given action to the OpenDSS model.
        """
        # Split action into respective components
        pv_kvar = action[:PV_KVAR_ACTION_LEN].copy()
        pv_kw = action[PV_KVAR_ACTION_LEN:PV_KVAR_ACTION_LEN + PV_KW_ACTION_LEN].copy()
        battery_kw = action[PV_KVAR_ACTION_LEN + PV_KW_ACTION_LEN:
                            PV_KVAR_ACTION_LEN + PV_KW_ACTION_LEN + BESS_KW_ACTION_LEN].copy()
        transformer_tap = action[PV_KVAR_ACTION_LEN + PV_KW_ACTION_LEN + BESS_KW_ACTION_LEN].copy()
        capacitor_states = action[PV_KVAR_ACTION_LEN + PV_KW_ACTION_LEN + BESS_KW_ACTION_LEN + TRANSFORMER_TAPS_ACTION_LEN:].copy()
        capacitor_states = np.round(capacitor_states)
        
        # Debugging: Print actions
        logger.debug(f"[ACTION] Applying Actions:")
        logger.debug(f"  PV_kVAR Actions: {pv_kvar}")
        logger.debug(f"  PV_kW Actions: {pv_kw}")
        logger.debug(f"  Battery_kW Actions: {battery_kw}")
        logger.debug(f"  Transformer Tap Action: {transformer_tap}")
        logger.debug(f"  Capacitor States: {capacitor_states}")
        
        # Retrieve current irradiance
        irradiance = self.irradiance_profile[self.current_step % len(self.irradiance_profile)]
        logger.debug(f"[INFO] Current Irradiance: {irradiance}")
        
        # Clip PV_kW based on irradiance
        for z in range(PV_KW_ACTION_LEN):
            if pv_kw[z] > irradiance * self.KWrated:
                logger.debug(f"[CLIP] PV_kW before clipping: {pv_kw[z]} > Irradiance * KWrated: {irradiance * self.KWrated}")
                pv_kw[z] = irradiance * self.KWrated
                logger.debug(f"[CLIP] PV_kW after clipping: {pv_kw[z]}")
        
        # Clip PV_kVAR based on maximum apparent power (S_max)
        S_max = self.KWrated  # Assuming KWrated represents S_max
        q_max1 = np.sqrt(S_max**2 - np.power(pv_kw, 2))
        pv_kvar = np.clip(pv_kvar, -q_max1, q_max1)
        logger.debug(f"[ACTION] PV_kVAR after clipping: {pv_kvar}")
        
        # Apply PV and Battery actions
        for i in range(min(PV_KW_ACTION_LEN, len(self.ranked_buses))):
            bus = self.ranked_buses[i]
            scaled_pv_kw = pv_kw[i]
            self.controller.text(f"edit PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR={pv_kvar[i]} Pmpp={scaled_pv_kw}")
            logger.debug(f"[CONTROL] Edited PVSystem.PV{i+1} on Bus {bus}: kVAR={pv_kvar[i]}, Pmpp={scaled_pv_kw}")
            
            self.controller.text(f"edit Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW={battery_kw[i]} kVAR=0")
            logger.debug(f"[CONTROL] Edited Storage.Battery{i+1} on Bus {bus}: kW={battery_kw[i]}, kVAR=0")
        
        # Apply Transformer Tap action
        self.controller.text(f"edit Transformer.reg1a taps={transformer_tap}")
        logger.debug(f"[CONTROL] Edited Transformer.reg1a: taps={transformer_tap}")
        
        # Apply Capacitor States
        for i, cap_name in enumerate(self.capacitor_names):
            if i < len(capacitor_states):
                state = int(capacitor_states[i])
                self.controller.text(f"edit Capacitor.{cap_name} states={state}")
                logger.debug(f"[CONTROL] Edited Capacitor.{cap_name}: states={state}")
        
        # Solve the power flow
        self.controller.text("set controlmode=off")
        self.controller.text("solve")
        logger.debug(f"[ACTION] Actions applied and OpenDSS solved.")
    
    def step(self, action):
        """
        Perform a single step in the environment.
        """
        logger.info(f"[STEP] Step {self.current_step}: Taking action.")
        self._take_action(action)
        
        # Retrieve bus voltages
        all_bus_voltages = self.controller.circuit_all_bus_vmag_pu()
        if isinstance(all_bus_voltages, list):
            pass  # Already a list of floats
        elif isinstance(all_bus_voltages, str):
            all_bus_voltages = [float(v) for v in all_bus_voltages.split()]
        else:
            raise TypeError(f"Unexpected type for bus voltages: {type(all_bus_voltages)}")
        
        # Verify observation length matches expected
        expected_length = self.observation_space.shape[0]
        if len(all_bus_voltages) != expected_length:
            logger.error(f"[ERROR] Observation length {len(all_bus_voltages)} does not match expected {expected_length}.")
            raise ValueError(f"Observation length {len(all_bus_voltages)} does not match expected {expected_length}.")
        
        # Calculate voltage violations
        voltage_violations = sum(1 for v in all_bus_voltages if (v <= 0.95 or v >= 1.05) and v != 0)
        logger.debug(f"[OBSERVATION] Bus Voltages: {all_bus_voltages}")
        logger.info(f"[REWARD] Voltage Violations: {voltage_violations}")
        
        # Define reward
        reward = -voltage_violations
        
        # Update logs
        self.hourly_violations_count.append(voltage_violations)
        self.all_bus_voltages.append(all_bus_voltages)
        
        # Get observation
        observation = self.get_observation()
        
        # Determine if the episode is done
        termination = (self.control_steps >= self.max_control_steps) or (voltage_violations == 0)
        self.control_steps += 1
        
        if termination:
            logger.info(f"[TERMINATION] Episode terminated at step {self.control_steps} with reward {reward}.")
        
        # Return observation, reward, done, truncated, info
        return observation, reward, termination, False, {}
    
    def reset(self, *, seed=None, options=None):
        """
        Reset the environment to an initial state.
        """
        if seed is not None:
            np.random.seed(seed)
        if options is not None:
            logger.info(f"[RESET] Reset options: {options}")
        
        # Recompile OpenDSS to reset the state using the stored dss_file path
        self.controller.text(f"compile [{self.dss_file}]")
        logger.info(f"[RESET] Recompiled OpenDSS file: {self.dss_file}")
        
        # Update the current step
        self.current_step = (self.current_step + 1) % len(self.irradiance_profile)
        self.control_steps = 0
        
        # Apply load scales
        load_scales = np.random.normal(loc=self.load_profile[self.current_step], scale=0.001, size=len(self.load_names))
        logger.info(f"[RESET] Applying Load Scales for step {self.current_step}.")
        
        for load_name, load_scale in zip(self.load_names, load_scales):
            load_power = self.load_powers[self.load_names.index(load_name)]
            result = load_power * load_scale * self.load_scale
            self.controller.text(f"edit Load.{load_name} kW={result}")
            logger.debug(f"[LOAD] Edited Load.{load_name}: kW={result}")
        
        # Apply PV and Battery initial states based on irradiance
        irradiance = self.irradiance_profile[self.current_step]
        logger.info(f"[RESET] Current Irradiance: {irradiance}")
        
        for i in range(min(PV_KW_ACTION_LEN, len(self.ranked_buses))):
            bus = self.ranked_buses[i]
            fixed_power_factor = 0.9
            pv_kw = 100 * irradiance
            power_factor_angle = np.arccos(fixed_power_factor)
            pv_kvar = pv_kw * np.tan(power_factor_angle)
            battery_kw = self.load_profile[self.current_step] * 100 * self.load_scale
            self.controller.text(f"edit PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR={pv_kvar} Pmpp={pv_kw}")
            logger.debug(f"[LOAD] Edited PVSystem.PV{i+1} on Bus {bus}: kVAR={pv_kvar}, Pmpp={pv_kw}")
            
            self.controller.text(f"edit Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW={battery_kw} kVAR=0")
            logger.debug(f"[LOAD] Edited Storage.Battery{i+1} on Bus {bus}: kW={battery_kw}, kVAR=0")
        
        # Solve the power flow after resetting
        self.controller.text("set controlmode=off")
        self.controller.text("solve")
        logger.info(f"[RESET] Load and PV/Battery states updated and OpenDSS solved.")
        
        # Get initial observation
        observation = self.get_observation()
        
        # Return the initial observation
        return observation, {}
    
    def get_observation(self):
        """
        Retrieve the current observation (bus voltages).
        """
        all_bus_voltages = self.controller.circuit_all_bus_vmag_pu()
        if isinstance(all_bus_voltages, list):
            pass  # Already a list of floats
        elif isinstance(all_bus_voltages, str):
            all_bus_voltages = [float(v) for v in all_bus_voltages.split()]
        else:
            raise TypeError(f"Unexpected type for bus voltages: {type(all_bus_voltages)}")
        
        observation = np.array(all_bus_voltages).astype(np.float32).flatten()
        
        # Clip values to ensure they are within the observation space bounds
        observation = np.clip(observation, self.observation_space.low, self.observation_space.high)
        
        logger.debug(f"[GET_OBSERVATION] Retrieved Bus Voltages: {observation}")
        return observation

# ------------------------------
# 2. Register the Environment
# ------------------------------

# Define the paths to your data files
dss_path = r"C:\Program Files\OpenDSS"  # Update if different
dss_file = r"C:\Users\Alaa\Documents\OpenDSS Model\OpenDSS Model (08.13.2020)\Master.dss"
irradiance_csv_file = r"D:\Alaa_Selim\Irradiance_Profile_Santa_Clara.csv"
load_profile_file = r"D:\Alaa_Selim\LoadShape1.csv"
load_powers_file = r"C:\Users\Alaa\Documents\OpenDSS Model\OpenDSS Model (08.13.2020)\Loaddemand.csv"

def env_creator(env_config):
    """
    Environment creator for registering with RLlib.
    """
    return PowerSystemEnv(
        dss_path=dss_path,
        dss_file=dss_file,
        irradiance_csv_file=irradiance_csv_file,
        load_profile_file=load_profile_file,
        load_powers_file=load_powers_file,
        seed=SEED,
        load_scale=5  # Adjust based on your system requirements
    )

# Register the environment with RLlib
register_env("PowerSystemEnv", env_creator)

# ------------------------------
# 3. Testing the Trained Agent
# ------------------------------

def test_trained_agent(checkpoint_path, num_episodes=10):
    """
    Test the trained IMPALA agent in the PowerSystemEnv environment.

    Parameters:
    - checkpoint_path (str): Path to the trained IMPALA checkpoint.
    - num_episodes (int): Number of episodes to run for testing.
    """
    # Initialize Ray
    ray.init(ignore_reinit_error=True)
    
    # Load the trained IMPALA agent from the checkpoint
    try:
        algo = Impala.from_checkpoint(checkpoint_path)
        logger.info(f"[TEST] Loaded IMPALA agent from checkpoint: {checkpoint_path}")
    except Exception as e:
        logger.error(f"[TEST] Failed to load IMPALA agent from checkpoint: {e}")
        ray.shutdown()
        return
    
    # Create an instance of the environment
    env = env_creator({})
    
    # Initialize lists to store results
    episode_rewards = []
    episode_lengths = []
    
    for episode in range(1, num_episodes + 1):
        try:
            obs, info = env.reset()
        except Exception as e:
            logger.error(f"[TEST] Failed to reset environment for Episode {episode}: {e}")
            continue
        
        done = False
        truncated = False
        total_reward = 0.0
        steps = 0
        
        while not (done or truncated):
            try:
                # Get action from the trained policy
                action_output = algo.get_policy().compute_single_action(obs)
                
                # Extract the action from the output
                if isinstance(action_output, tuple):
                    action = action_output[0]
                else:
                    action = action_output
                
                # Ensure the action is a NumPy array
                if not isinstance(action, np.ndarray):
                    action = np.array(action)
                
                # Take the action in the environment
                obs, reward, done, truncated, info = env.step(action)
                
                # Accumulate reward and steps
                total_reward += reward
                steps += 1
            except Exception as e:
                logger.error(f"[TEST] Error during Episode {episode}, Step {steps}: {e}")
                break
        
        episode_rewards.append(total_reward)
        episode_lengths.append(steps)
        
        print(f"Episode {episode}: Total Reward = {total_reward}, Steps = {steps}")
    
    # Calculate and display average metrics
    avg_reward = np.mean(episode_rewards)
    avg_length = np.mean(episode_lengths)
    
    print("\n=== Testing Results ===")
    print(f"Number of Episodes: {num_episodes}")
    print(f"Average Total Reward: {avg_reward}")
    print(f"Average Episode Length: {avg_length}")
    
    # Shutdown Ray
    ray.shutdown()

# ------------------------------
# 4. Main Execution
# ------------------------------

if __name__ == "__main__":
    # Define the path to your trained IMPALA checkpoint
    checkpoint_path = r"C:\Users\Alaa\ray_results\IMPALA\IMPALA_PowerSystemEnv_f6ecd_00000_0_lr=0.0001_2024-11-03_22-14-55\checkpoint_000020"
    
    if not os.path.exists(checkpoint_path):
        print(f"Checkpoint path does not exist: {checkpoint_path}")
    else:
        # Number of test episodes
        num_test_episodes = 4  # Adjust as needed
        
        # Run the testing function
        test_trained_agent(checkpoint_path, num_episodes=num_test_episodes)


In [ ]:
import os
import ray
from ray.rllib.algorithms.impala import Impala
from ray.tune.registry import register_env
import gymnasium as gym
from gymnasium import spaces
import numpy as np
from py_dss_interface import DSSDLL
import pandas as pd
import torch
import warnings
import logging
import csv

# ------------------------------
# 1. Environment Definition
# ------------------------------

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Configure logging
logging.basicConfig(level=logging.INFO)  # Set to DEBUG for more detailed logs
logger = logging.getLogger(__name__)

# Define lengths for each segment of the action space
PV_KVAR_ACTION_LEN = 30          # Number of PV systems for reactive power control
PV_KW_ACTION_LEN = 30            # Number of PV systems for active power control
BESS_KW_ACTION_LEN = 30          # Number of Batteries for active power control
TRANSFORMER_TAPS_ACTION_LEN = 1  # Number of transformer taps
CAPACITOR_ACTION_LEN = 2         # Number of Capacitors (Manually defined)

# Define action value ranges for each controllable element
PV_KVAR_ACTION_LOW = -800
PV_KVAR_ACTION_HIGH = 800
PV_KW_ACTION_LOW = 0
PV_KW_ACTION_HIGH = 1000
BESS_KW_ACTION_LOW = 0
BESS_KW_ACTION_HIGH = 1000
TRANSFORMER_TAPS_ACTION_LOW = 0.9
TRANSFORMER_TAPS_ACTION_HIGH = 1.1
CAPACITOR_ACTION_LOW = 0
CAPACITOR_ACTION_HIGH = 1

# Set a seed for reproducibility
SEED = 42

class PowerSystemEnv(gym.Env):
    """
    Custom Environment for Power System Control using OpenDSS and Gymnasium.
    """
    metadata = {'render.modes': ['human'], 'spec.max_episode_steps': 1000}

    def __init__(self, dss_path, dss_file, irradiance_csv_file, load_profile_file, load_powers_file, seed=None, load_scale=3):
        super(PowerSystemEnv, self).__init__()
        self.seed(seed)
        
        # Store the OpenDSS file path for later use
        self.dss_file = dss_file
        
        # Initialize OpenDSS Controller
        self.controller = DSSDLL(dss_path)
        self.controller.text(f"compile [{dss_file}]")
        logger.info(f"[INIT] Compiled OpenDSS file: {dss_file}")
        
        # Manually define capacitor names
        self.capacitor_names = ["CAP_201", "CAP_301"]
        logger.info(f"[INIT] Defined capacitors: {self.capacitor_names}")
        
        # Retrieve bus names from OpenDSS
        self.ranked_buses = self.get_bus_names()
        logger.info(f"[INIT] Retrieved {len(self.ranked_buses)} buses from OpenDSS.")
        
        # Manually define load names
        self.load_names = [
            "Load_1003", "Load_1004", "Load_1005", "Load_1006", "Load_1007", "Load_1008",
            "Load_1009", "Load_1010", "Load_1011", "Load_1012", "Load_1013", "Load_1014",
            "Load_1015", "Load_1016", "Load_1017", "Load_2002", "Load_2003", "Load_2005",
            "Load_2008", "Load_2009", "Load_2010", "Load_2011", "Load_2014", "Load_2015",
            "Load_2016", "Load_2017", "Load_2018", "Load_2020", "Load_2022", "Load_2023",
            "Load_2024", "Load_2025", "Load_2028", "Load_2029", "Load_2030", "Load_2031",
            "Load_2032", "Load_2034", "Load_2035", "Load_2037", "Load_2040", "Load_2041",
            "Load_2042", "Load_2043", "Load_2045", "Load_2046", "Load_2047", "Load_2048",
            "Load_2049", "Load_2050", "Load_2051", "Load_2052", "Load_2053", "Load_2054",
            "Load_2055", "Load_2056", "Load_2058", "Load_2059", "Load_2060",
            "Load_3002", "Load_3004", "Load_3006", "Load_3007", "Load_3009", "Load_3010",
            "Load_3011", "Load_3012", "Load_3013", "Load_3014", "Load_3016", "Load_3017",
            "Load_3018", "Load_3019", "Load_3020", "Load_3021", "Load_3023", "Load_3024",
            "Load_3025", "Load_3026", "Load_3027", "Load_3028", "Load_3029", "Load_3031",
            "Load_3032", "Load_3033", "Load_3034", "Load_3035", "Load_3036", "Load_3037",
            "Load_3038", "Load_3039", "Load_3041", "Load_3042", "Load_3043", "Load_3044",
            "Load_3045", "Load_3047", "Load_3048", "Load_3049", "Load_3050", "Load_3051",
            "Load_3052", "Load_3054", "Load_3056", "Load_3057", "Load_3058", "Load_3059",
            "Load_3060", "Load_3061", "Load_3062", "Load_3063", "Load_3064", "Load_3065",
            "Load_3066", "Load_3067", "Load_3070", "Load_3071", "Load_3072", "Load_3073",
            "Load_3074", "Load_3077", "Load_3078", "Load_3081", "Load_3083", "Load_3084",
            "Load_3085", "Load_3086", "Load_3087", "Load_3088", "Load_3089", "Load_3090",
            "Load_3091", "Load_3093", "Load_3094", "Load_3095", "Load_3096", "Load_3097",
            "Load_3098", "Load_3099", "Load_3101", "Load_3102", "Load_3103", "Load_3104",
            "Load_3105", "Load_3106", "Load_3108", "Load_3109", "Load_3110", "Load_3111",
            "Load_3112", "Load_3114", "Load_3115", "Load_3116", "Load_3117", "Load_3120",
            "Load_3121", "Load_3122", "Load_3123", "Load_3124", "Load_3125", "Load_3126",
            "Load_3127", "Load_3128", "Load_3129", "Load_3130", "Load_3131", "Load_3132",
            "Load_3134", "Load_3135", "Load_3136", "Load_3137", "Load_3138", "Load_3139",
            "Load_3141", "Load_3142", "Load_3143", "Load_3144", "Load_3145", "Load_3146",
            "Load_3147", "Load_3148", "Load_3149", "Load_3150", "Load_3151", "Load_3152",
            "Load_3153", "Load_3154", "Load_3155", "Load_3157", "Load_3158", "Load_3159",
            "Load_3160", "Load_3161", "Load_3162"
        ]
        logger.info(f"[INIT] Defined loads: {len(self.load_names)} loads loaded.")
        
        # Validate load names against OpenDSS
        self.validate_loads()
        
        # Define system parameters
        self.KWrated = 1000
        self.load_scale = load_scale
        
        # Initialize PV systems and Batteries
        num_pv = PV_KW_ACTION_LEN  # 30 PV systems
        num_battery = BESS_KW_ACTION_LEN  # 30 Batteries
        logger.info(f"[INIT] Initializing {num_pv} PV systems and {num_battery} Batteries.")
        
        for i in range(num_pv):
            if i < len(self.ranked_buses):
                bus = self.ranked_buses[i]
                self.controller.text(f"new PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR=0 KVA=100 Pmpp=80")
                logger.debug(f"[INIT] Initialized PVSystem.PV{i+1} on Bus {bus}")
            else:
                logger.warning(f"[INIT] Not enough buses to initialize PVSystem.PV{i+1}")
        
        for i in range(num_battery):
            if i < len(self.ranked_buses):
                bus = self.ranked_buses[i]
                self.controller.text(f"new Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW=100 kVAR=0")
                logger.debug(f"[INIT] Initialized Storage.Battery{i+1} on Bus {bus}")
            else:
                logger.warning(f"[INIT] Not enough buses to initialize Storage.Battery{i+1}")
        
        # Load profiles
        self.irradiance_profile = self.load_csv(irradiance_csv_file, "Irradiance")
        self.load_profile = self.load_csv(load_profile_file, "Load Profile", skip_header=True)
        self.load_powers = self.load_csv(load_powers_file, "Load Powers", skip_header=False)
        
        # Validate load powers
        if len(self.load_powers) != len(self.load_names):
            raise ValueError("Number of load powers does not match number of load names.")
        logger.info("[INIT] Load powers and load names are consistent.")
        
        # Define action and observation spaces
        self.action_space = spaces.Box(
            low=np.array(
                [PV_KVAR_ACTION_LOW] * PV_KVAR_ACTION_LEN +
                [PV_KW_ACTION_LOW] * PV_KW_ACTION_LEN +
                [BESS_KW_ACTION_LOW] * BESS_KW_ACTION_LEN +
                [TRANSFORMER_TAPS_ACTION_LOW] * TRANSFORMER_TAPS_ACTION_LEN +
                [CAPACITOR_ACTION_LOW] * CAPACITOR_ACTION_LEN
            ),
            high=np.array(
                [PV_KVAR_ACTION_HIGH] * PV_KVAR_ACTION_LEN +
                [PV_KW_ACTION_HIGH] * PV_KW_ACTION_LEN +
                [BESS_KW_ACTION_HIGH] * BESS_KW_ACTION_LEN +
                [TRANSFORMER_TAPS_ACTION_HIGH] * TRANSFORMER_TAPS_ACTION_LEN +
                [CAPACITOR_ACTION_HIGH] * CAPACITOR_ACTION_LEN
            ),
            dtype=np.float32
        )
        logger.info(f"[INIT] Defined action space with shape: {self.action_space.shape}")
        
        # Update the number of observations based on current setup
        num_buses = len(self.ranked_buses)
        expected_observations = 915
        if num_buses != expected_observations:
            logger.warning(f"[INIT] Number of buses ({num_buses}) does not match expected observations ({expected_observations}). Please verify.")
        
        self.observation_space = spaces.Box(
            low=0.0,
            high=2.0,
            shape=(expected_observations,),
            dtype=np.float32
        )
        logger.info(f"[INIT] Defined observation space with shape: {self.observation_space.shape}")
        
        # Initialize other variables
        self.current_step = 0
        self.control_steps = 0
        self.max_control_steps = 1000  # Define as needed
        self.hourly_violations_count = []
        self.all_bus_voltages = []
    
    def load_csv(self, file_path, description, skip_header=False):
        """
        Utility method to load CSV files.
        """
        try:
            with open(file_path, 'r') as csvfile:
                reader = csv.reader(csvfile)
                if skip_header:
                    next(reader, None)  # Skip header if present
                data = [float(row[0]) for row in reader]
            logger.info(f"[LOAD] Loaded {description} with {len(data)} entries.")
            return data
        except Exception as e:
            logger.error(f"[LOAD] Failed to load {description} from {file_path}: {e}")
            raise e
    
    def validate_loads(self):
        """
        Validates that all load names exist in the OpenDSS model.
        """
        # Since 'circuit_all_load_names()' is unavailable, manual validation is limited.
        # Ensure that the load names in self.load_names match those in OpenDSS.
        # Alternatively, proceed with caution.
        logger.info("[VALIDATE] Manual validation of load names is required.")
    
    def seed(self, seed=None):
        """
        Set the seed for reproducibility.
        """
        self.seed_value = seed
        np.random.seed(seed)
        torch.manual_seed(seed)
        if seed is not None:
            torch.cuda.manual_seed_all(seed)
        logger.info(f"[SEED] Environment seeded with value: {seed}")
    
    def get_bus_names(self):
        """
        Retrieve all bus names from OpenDSS.
        """
        bus_names = self.controller.circuit_all_bus_names()
        if isinstance(bus_names, list):
            logger.info(f"[INFO] Retrieved {len(bus_names)} bus names.")
            return bus_names
        else:
            raise TypeError(f"Expected list for bus names, got {type(bus_names)}")
    
    def _take_action(self, action):
        """
        Apply the given action to the OpenDSS model.
        """
        # Split action into respective components
        pv_kvar = action[:PV_KVAR_ACTION_LEN].copy()
        pv_kw = action[PV_KVAR_ACTION_LEN:PV_KVAR_ACTION_LEN + PV_KW_ACTION_LEN].copy()
        battery_kw = action[PV_KVAR_ACTION_LEN + PV_KW_ACTION_LEN:
                            PV_KVAR_ACTION_LEN + PV_KW_ACTION_LEN + BESS_KW_ACTION_LEN].copy()
        transformer_tap = action[PV_KVAR_ACTION_LEN + PV_KW_ACTION_LEN + BESS_KW_ACTION_LEN].copy()
        capacitor_states = action[PV_KVAR_ACTION_LEN + PV_KW_ACTION_LEN + BESS_KW_ACTION_LEN + TRANSFORMER_TAPS_ACTION_LEN:].copy()
        capacitor_states = np.round(capacitor_states)
        
        # Debugging: Print actions
        logger.debug(f"[ACTION] Applying Actions:")
        logger.debug(f"  PV_kVAR Actions: {pv_kvar}")
        logger.debug(f"  PV_kW Actions: {pv_kw}")
        logger.debug(f"  Battery_kW Actions: {battery_kw}")
        logger.debug(f"  Transformer Tap Action: {transformer_tap}")
        logger.debug(f"  Capacitor States: {capacitor_states}")
        
        # Retrieve current irradiance
        irradiance = self.irradiance_profile[self.current_step % len(self.irradiance_profile)]
        logger.debug(f"[INFO] Current Irradiance: {irradiance}")
        
        # Clip PV_kW based on irradiance
        for z in range(PV_KW_ACTION_LEN):
            if pv_kw[z] > irradiance * self.KWrated:
                logger.debug(f"[CLIP] PV_kW before clipping: {pv_kw[z]} > Irradiance * KWrated: {irradiance * self.KWrated}")
                pv_kw[z] = irradiance * self.KWrated
                logger.debug(f"[CLIP] PV_kW after clipping: {pv_kw[z]}")
        
        # Clip PV_kVAR based on maximum apparent power (S_max)
        S_max = self.KWrated  # Assuming KWrated represents S_max
        q_max1 = np.sqrt(S_max**2 - np.power(pv_kw, 2))
        pv_kvar = np.clip(pv_kvar, -q_max1, q_max1)
        logger.debug(f"[ACTION] PV_kVAR after clipping: {pv_kvar}")
        
        # Apply PV and Battery actions
        for i in range(min(PV_KW_ACTION_LEN, len(self.ranked_buses))):
            bus = self.ranked_buses[i]
            scaled_pv_kw = pv_kw[i]
            self.controller.text(f"edit PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR={pv_kvar[i]} Pmpp={scaled_pv_kw}")
            logger.debug(f"[CONTROL] Edited PVSystem.PV{i+1} on Bus {bus}: kVAR={pv_kvar[i]}, Pmpp={scaled_pv_kw}")
            
            self.controller.text(f"edit Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW={battery_kw[i]} kVAR=0")
            logger.debug(f"[CONTROL] Edited Storage.Battery{i+1} on Bus {bus}: kW={battery_kw[i]}, kVAR=0")
        
        # Apply Transformer Tap action
        self.controller.text(f"edit Transformer.reg1a taps={transformer_tap}")
        logger.debug(f"[CONTROL] Edited Transformer.reg1a: taps={transformer_tap}")
        
        # Apply Capacitor States
        for i, cap_name in enumerate(self.capacitor_names):
            if i < len(capacitor_states):
                state = int(capacitor_states[i])
                self.controller.text(f"edit Capacitor.{cap_name} states={state}")
                logger.debug(f"[CONTROL] Edited Capacitor.{cap_name}: states={state}")
        
        # Solve the power flow
        self.controller.text("set controlmode=off")
        self.controller.text("solve")
        logger.debug(f"[ACTION] Actions applied and OpenDSS solved.")
    
    def step(self, action):
        """
        Perform a single step in the environment.
        """
        logger.info(f"[STEP] Step {self.current_step}: Taking action.")
        self._take_action(action)
        
        # Retrieve bus voltages
        all_bus_voltages = self.controller.circuit_all_bus_vmag_pu()
        if isinstance(all_bus_voltages, list):
            pass  # Already a list of floats
        elif isinstance(all_bus_voltages, str):
            all_bus_voltages = [float(v) for v in all_bus_voltages.split()]
        else:
            raise TypeError(f"Unexpected type for bus voltages: {type(all_bus_voltages)}")
        
        # Verify observation length matches expected
        expected_length = self.observation_space.shape[0]
        if len(all_bus_voltages) != expected_length:
            logger.error(f"[ERROR] Observation length {len(all_bus_voltages)} does not match expected {expected_length}.")
            raise ValueError(f"Observation length {len(all_bus_voltages)} does not match expected {expected_length}.")
        
        # Calculate voltage violations
        voltage_violations = sum(1 for v in all_bus_voltages if (v <= 0.95 or v >= 1.05) and v != 0)
        logger.debug(f"[OBSERVATION] Bus Voltages: {all_bus_voltages}")
        logger.info(f"[REWARD] Voltage Violations: {voltage_violations}")
        
        # Define reward
        reward = -voltage_violations
        
        # Update logs
        self.hourly_violations_count.append(voltage_violations)
        self.all_bus_voltages.append(all_bus_voltages)
        
        # Get observation
        observation = self.get_observation()
        
        # Determine if the episode is done
        termination = (self.control_steps >= self.max_control_steps) or (voltage_violations == 0)
        self.control_steps += 1
        
        if termination:
            logger.info(f"[TERMINATION] Episode terminated at step {self.control_steps} with reward {reward}.")
        
        # Return observation, reward, done, truncated, info
        return observation, reward, termination, False, {}
    
    def reset(self, *, seed=None, options=None):
        """
        Reset the environment to an initial state.
        """
        if seed is not None:
            np.random.seed(seed)
        if options is not None:
            logger.info(f"[RESET] Reset options: {options}")
        
        # Recompile OpenDSS to reset the state using the stored dss_file path
        self.controller.text(f"compile [{self.dss_file}]")
        logger.info(f"[RESET] Recompiled OpenDSS file: {self.dss_file}")
        
        # Update the current step
        self.current_step = (self.current_step + 1) % len(self.irradiance_profile)
        self.control_steps = 0
        
        # Apply load scales
        load_scales = np.random.normal(loc=self.load_profile[self.current_step], scale=0.001, size=len(self.load_names))
        logger.info(f"[RESET] Applying Load Scales for step {self.current_step}.")
        
        for load_name, load_scale in zip(self.load_names, load_scales):
            load_power = self.load_powers[self.load_names.index(load_name)]
            result = load_power * load_scale * self.load_scale
            self.controller.text(f"edit Load.{load_name} kW={result}")
            logger.debug(f"[LOAD] Edited Load.{load_name}: kW={result}")
        
        # Apply PV and Battery initial states based on irradiance
        irradiance = self.irradiance_profile[self.current_step]
        logger.info(f"[RESET] Current Irradiance: {irradiance}")
        
        for i in range(min(PV_KW_ACTION_LEN, len(self.ranked_buses))):
            bus = self.ranked_buses[i]
            fixed_power_factor = 0.9
            pv_kw = 100 * irradiance
            power_factor_angle = np.arccos(fixed_power_factor)
            pv_kvar = pv_kw * np.tan(power_factor_angle)
            battery_kw = self.load_profile[self.current_step] * 100 * self.load_scale
            self.controller.text(f"edit PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR={pv_kvar} Pmpp={pv_kw}")
            logger.debug(f"[LOAD] Edited PVSystem.PV{i+1} on Bus {bus}: kVAR={pv_kvar}, Pmpp={pv_kw}")
            
            self.controller.text(f"edit Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW={battery_kw} kVAR=0")
            logger.debug(f"[LOAD] Edited Storage.Battery{i+1} on Bus {bus}: kW={battery_kw}, kVAR=0")
        
        # Solve the power flow after resetting
        self.controller.text("set controlmode=off")
        self.controller.text("solve")
        logger.info(f"[RESET] Load and PV/Battery states updated and OpenDSS solved.")
        
        # Get initial observation
        observation = self.get_observation()
        
        # Return the initial observation
        return observation, {}
    
    def get_observation(self):
        """
        Retrieve the current observation (bus voltages).
        """
        all_bus_voltages = self.controller.circuit_all_bus_vmag_pu()
        if isinstance(all_bus_voltages, list):
            pass  # Already a list of floats
        elif isinstance(all_bus_voltages, str):
            all_bus_voltages = [float(v) for v in all_bus_voltages.split()]
        else:
            raise TypeError(f"Unexpected type for bus voltages: {type(all_bus_voltages)}")
        
        observation = np.array(all_bus_voltages).astype(np.float32).flatten()
        
        # Clip values to ensure they are within the observation space bounds
        observation = np.clip(observation, self.observation_space.low, self.observation_space.high)
        
        logger.debug(f"[GET_OBSERVATION] Retrieved Bus Voltages: {observation}")
        return observation

# ------------------------------
# 2. Register the Environment
# ------------------------------

# Define the paths to your data files
dss_path = r"C:\Program Files\OpenDSS"  # Update if different
dss_file = r"C:\Users\Alaa\Documents\OpenDSS Model\OpenDSS Model (08.13.2020)\Master.dss"
irradiance_csv_file = r"D:\Alaa_Selim\Irradiance_Profile_Santa_Clara.csv"
load_profile_file = r"D:\Alaa_Selim\LoadShape1.csv"
load_powers_file = r"C:\Users\Alaa\Documents\OpenDSS Model\OpenDSS Model (08.13.2020)\Loaddemand.csv"

def env_creator(env_config):
    """
    Environment creator for registering with RLlib.
    """
    return PowerSystemEnv(
        dss_path=dss_path,
        dss_file=dss_file,
        irradiance_csv_file=irradiance_csv_file,
        load_profile_file=load_profile_file,
        load_powers_file=load_powers_file,
        seed=SEED,
        load_scale=10  # Adjust based on your system requirements
    )

# Register the environment with RLlib
register_env("PowerSystemEnv", env_creator)

# ------------------------------
# 3. Baseline Policy Definition
# ------------------------------

def baseline_policy(action_space):
    """
    Generate a baseline action.
    For this example, the baseline action is to set all controllable elements to zero.
    Modify this function to implement more sophisticated baseline strategies if desired.
    
    Parameters:
    - action_space: The action space of the environment.
    
    Returns:
    - action: A NumPy array representing the baseline action.
    """
    # Initialize action array with zeros
    action = np.zeros(action_space.shape, dtype=np.float32)
    
    # Optionally, implement more complex baseline logic here
    # For example, keep transformer taps at default, capacitors off, etc.
    
    return action

# ------------------------------
# 4. Testing Functions
# ------------------------------

def test_drl_agent(algo, env, num_episodes=10):
    """
    Test the trained DRL agent in the environment.
    
    Parameters:
    - algo: The trained IMPALA algorithm instance.
    - env: The environment instance.
    - num_episodes: Number of episodes to run.
    
    Returns:
    - rewards: List of total rewards per episode.
    - lengths: List of episode lengths.
    """
    rewards = []
    lengths = []
    
    for episode in range(1, num_episodes + 1):
        try:
            obs, info = env.reset()
        except Exception as e:
            logger.error(f"[DRL TEST] Failed to reset environment for Episode {episode}: {e}")
            continue
        
        done = False
        truncated = False
        total_reward = 0.0
        steps = 0
        
        while not (done or truncated):
            try:
                # Get action from the trained policy
                action_output = algo.get_policy().compute_single_action(obs)
                
                # Extract the action from the output
                if isinstance(action_output, tuple):
                    action = action_output[0]
                else:
                    action = action_output
                
                # Ensure the action is a NumPy array
                if not isinstance(action, np.ndarray):
                    action = np.array(action)
                
                # Take the action in the environment
                obs, reward, done, truncated, info = env.step(action)
                
                # Accumulate reward and steps
                total_reward += reward
                steps += 1
            except Exception as e:
                logger.error(f"[DRL TEST] Error during Episode {episode}, Step {steps}: {e}")
                break
        
        rewards.append(total_reward)
        lengths.append(steps)
        
        print(f"DRL Agent - Episode {episode}: Total Reward = {total_reward}, Steps = {steps}")
    
    return rewards, lengths

def test_baseline_policy(env, num_episodes=10):
    """
    Test the baseline (non-control) policy in the environment.
    
    Parameters:
    - env: The environment instance.
    - num_episodes: Number of episodes to run.
    
    Returns:
    - rewards: List of total rewards per episode.
    - lengths: List of episode lengths.
    """
    rewards = []
    lengths = []
    
    for episode in range(1, num_episodes + 1):
        try:
            obs, info = env.reset()
        except Exception as e:
            logger.error(f"[BASELINE TEST] Failed to reset environment for Episode {episode}: {e}")
            continue
        
        done = False
        truncated = False
        total_reward = 0.0
        steps = 0
        
        while not (done or truncated):
            try:
                # Get baseline action
                action = baseline_policy(env.action_space)
                
                # Take the action in the environment
                obs, reward, done, truncated, info = env.step(action)
                
                # Accumulate reward and steps
                total_reward += reward
                steps += 1
            except Exception as e:
                logger.error(f"[BASELINE TEST] Error during Episode {episode}, Step {steps}: {e}")
                break
        
        rewards.append(total_reward)
        lengths.append(steps)
        
        print(f"Baseline Policy - Episode {episode}: Total Reward = {total_reward}, Steps = {steps}")
    
    return rewards, lengths

def compare_policies(drl_rewards, drl_lengths, baseline_rewards, baseline_lengths):
    """
    Compare the performance of DRL agent and baseline policy.
    
    Parameters:
    - drl_rewards: List of DRL agent rewards.
    - drl_lengths: List of DRL agent episode lengths.
    - baseline_rewards: List of baseline policy rewards.
    - baseline_lengths: List of baseline policy episode lengths.
    """
    print("\n=== Performance Comparison ===")
    
    # DRL Agent Metrics
    drl_avg_reward = np.mean(drl_rewards)
    drl_avg_length = np.mean(drl_lengths)
    
    # Baseline Policy Metrics
    baseline_avg_reward = np.mean(baseline_rewards)
    baseline_avg_length = np.mean(baseline_lengths)
    
    print(f"DRL Agent - Average Total Reward: {drl_avg_reward}, Average Episode Length: {drl_avg_length}")
    print(f"Baseline Policy - Average Total Reward: {baseline_avg_reward}, Average Episode Length: {baseline_avg_length}")
    
    # Optionally, add more comparison metrics or statistical tests here

# ------------------------------
# 5. Main Execution
# ------------------------------

def main():
    # Initialize Ray
    ray.init(ignore_reinit_error=True)
    
    # Define the path to your trained IMPALA checkpoint
    checkpoint_path = r"C:\Users\Alaa\ray_results\IMPALA\IMPALA_PowerSystemEnv_f6ecd_00000_0_lr=0.0001_2024-11-03_22-14-55\checkpoint_000020"
    
    if not os.path.exists(checkpoint_path):
        print(f"Checkpoint path does not exist: {checkpoint_path}")
        ray.shutdown()
        return
    
    # Load the trained IMPALA agent from the checkpoint
    try:
        algo = Impala.from_checkpoint(checkpoint_path)
        logger.info(f"[MAIN] Loaded IMPALA agent from checkpoint: {checkpoint_path}")
    except Exception as e:
        logger.error(f"[MAIN] Failed to load IMPALA agent from checkpoint: {e}")
        ray.shutdown()
        return
    
    # Create an instance of the environment
    env = env_creator({})
    
    # Define number of test episodes
    num_test_episodes = 10  # Adjust as needed
    
    # Test the DRL agent
    print("\n--- Testing DRL Agent ---")
    drl_rewards, drl_lengths = test_drl_agent(algo, env, num_test_episodes)
    
    # Test the Baseline Policy
    print("\n--- Testing Baseline Policy (No Control) ---")
    baseline_rewards, baseline_lengths = test_baseline_policy(env, num_test_episodes)
    
    # Compare the performance
    compare_policies(drl_rewards, drl_lengths, baseline_rewards, baseline_lengths)
    
    # Shutdown Ray
    ray.shutdown()

if __name__ == "__main__":
    main()


2024-11-07 12:00:45,359	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
2024-11-07 12:01:14,383	INFO worker.py:1621 -- Started a local Ray instance.
2024-11-07 12:01:17,247	WARNING algorithm_config.py:656 -- Cannot create ImpalaConfig from given `config_dict`! Property __stdout_file__ not supported.
(pid=134884) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


(RolloutWorker pid=134884) OpenDSS Started successfully! 
(RolloutWorker pid=134884) OpenDSS Version 9.5.1.1 (64-bit build); License Status: Open 
(RolloutWorker pid=134884) 
(RolloutWorker pid=134884) 


(RolloutWorker pid=130828) [INIT] Number of buses (436) does not match expected observations (915). Please verify.
(RolloutWorker pid=130828) 2024-11-07 12:01:20,844	INFO policy.py:1294 -- Policy (worker=3) running on CPU.
(RolloutWorker pid=130828) 2024-11-07 12:01:20,844	INFO torch_policy_v2.py:113 -- Found 1 visible cuda devices.
(RolloutWorker pid=130828) 2024-11-07 12:01:20,863	INFO util.py:118 -- Using connectors:
(RolloutWorker pid=130828) 2024-11-07 12:01:20,864	INFO util.py:119 --     AgentConnectorPipeline
(RolloutWorker pid=130828)         ObsPreprocessorConnector
(RolloutWorker pid=130828)         StateBufferConnector
(RolloutWorker pid=130828)         ViewRequirementAgentConnector
(RolloutWorker pid=130828) 2024-11-07 12:01:20,864	INFO util.py:120 --     ActionConnectorPipeline
(RolloutWorker pid=130828)         ConvertToNumpyConnector
(RolloutWorker pid=130828)         NormalizeActionsConnector
(RolloutWorker pid=130828)         ImmutableActionsConnector


In [1]:
import os
import ray
from ray.rllib.algorithms.sac import SAC
from ray.tune.registry import register_env
import gymnasium as gym
from gymnasium import spaces
import numpy as np
from py_dss_interface import DSSDLL
import pandas as pd
import torch
import warnings
import logging
import csv

# ------------------------------
# 1. Environment Definition
# ------------------------------

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Configure logging
logging.basicConfig(level=logging.INFO)  # Set to DEBUG for more detailed logs
logger = logging.getLogger(__name__)

# Define lengths for each segment of the action space
PV_KVAR_ACTION_LEN = 30          # Number of PV systems for reactive power control
PV_KW_ACTION_LEN = 30            # Number of PV systems for active power control
BESS_KW_ACTION_LEN = 30          # Number of Batteries for active power control
TRANSFORMER_TAPS_ACTION_LEN = 1  # Number of transformer taps
CAPACITOR_ACTION_LEN = 2         # Number of Capacitors (Manually defined)

# Define action value ranges for each controllable element
PV_KVAR_ACTION_LOW = -800
PV_KVAR_ACTION_HIGH = 800
PV_KW_ACTION_LOW = 0
PV_KW_ACTION_HIGH = 1000
BESS_KW_ACTION_LOW = 0
BESS_KW_ACTION_HIGH = 1000
TRANSFORMER_TAPS_ACTION_LOW = 0.9
TRANSFORMER_TAPS_ACTION_HIGH = 1.1
CAPACITOR_ACTION_LOW = 0
CAPACITOR_ACTION_HIGH = 1

# Set a seed for reproducibility
SEED = 42

class PowerSystemEnv(gym.Env):
    """
    Custom Environment for Power System Control using OpenDSS and Gymnasium.
    """
    metadata = {'render.modes': ['human'], 'spec.max_episode_steps': 1000}

    def __init__(self, dss_path, dss_file, irradiance_csv_file, load_profile_file, load_powers_file, seed=None, load_scale=3):
        super(PowerSystemEnv, self).__init__()
        self.seed(seed)
        
        # Store the OpenDSS file path for later use
        self.dss_file = dss_file
        
        # Initialize OpenDSS Controller
        self.controller = DSSDLL(dss_path)
        self.controller.text(f"compile [{dss_file}]")
        logger.info(f"[INIT] Compiled OpenDSS file: {dss_file}")
        
        # Manually define capacitor names
        self.capacitor_names = ["CAP_201", "CAP_301"]
        logger.info(f"[INIT] Defined capacitors: {self.capacitor_names}")
        
        # Retrieve bus names from OpenDSS
        self.ranked_buses = self.get_bus_names()
        logger.info(f"[INIT] Retrieved {len(self.ranked_buses)} buses from OpenDSS.")
        
        # Manually define load names
        self.load_names = [
            "Load_1003", "Load_1004", "Load_1005", "Load_1006", "Load_1007", "Load_1008",
            "Load_1009", "Load_1010", "Load_1011", "Load_1012", "Load_1013", "Load_1014",
            "Load_1015", "Load_1016", "Load_1017", "Load_2002", "Load_2003", "Load_2005",
            "Load_2008", "Load_2009", "Load_2010", "Load_2011", "Load_2014", "Load_2015",
            "Load_2016", "Load_2017", "Load_2018", "Load_2020", "Load_2022", "Load_2023",
            "Load_2024", "Load_2025", "Load_2028", "Load_2029", "Load_2030", "Load_2031",
            "Load_2032", "Load_2034", "Load_2035", "Load_2037", "Load_2040", "Load_2041",
            "Load_2042", "Load_2043", "Load_2045", "Load_2046", "Load_2047", "Load_2048",
            "Load_2049", "Load_2050", "Load_2051", "Load_2052", "Load_2053", "Load_2054",
            "Load_2055", "Load_2056", "Load_2058", "Load_2059", "Load_2060",
            "Load_3002", "Load_3004", "Load_3006", "Load_3007", "Load_3009", "Load_3010",
            "Load_3011", "Load_3012", "Load_3013", "Load_3014", "Load_3016", "Load_3017",
            "Load_3018", "Load_3019", "Load_3020", "Load_3021", "Load_3023", "Load_3024",
            "Load_3025", "Load_3026", "Load_3027", "Load_3028", "Load_3029", "Load_3031",
            "Load_3032", "Load_3033", "Load_3034", "Load_3035", "Load_3036", "Load_3037",
            "Load_3038", "Load_3039", "Load_3041", "Load_3042", "Load_3043", "Load_3044",
            "Load_3045", "Load_3047", "Load_3048", "Load_3049", "Load_3050", "Load_3051",
            "Load_3052", "Load_3054", "Load_3056", "Load_3057", "Load_3058", "Load_3059",
            "Load_3060", "Load_3061", "Load_3062", "Load_3063", "Load_3064", "Load_3065",
            "Load_3066", "Load_3067", "Load_3070", "Load_3071", "Load_3072", "Load_3073",
            "Load_3074", "Load_3077", "Load_3078", "Load_3081", "Load_3083", "Load_3084",
            "Load_3085", "Load_3086", "Load_3087", "Load_3088", "Load_3089", "Load_3090",
            "Load_3091", "Load_3093", "Load_3094", "Load_3095", "Load_3096", "Load_3097",
            "Load_3098", "Load_3099", "Load_3101", "Load_3102", "Load_3103", "Load_3104",
            "Load_3105", "Load_3106", "Load_3108", "Load_3109", "Load_3110", "Load_3111",
            "Load_3112", "Load_3114", "Load_3115", "Load_3116", "Load_3117", "Load_3120",
            "Load_3121", "Load_3122", "Load_3123", "Load_3124", "Load_3125", "Load_3126",
            "Load_3127", "Load_3128", "Load_3129", "Load_3130", "Load_3131", "Load_3132",
            "Load_3134", "Load_3135", "Load_3136", "Load_3137", "Load_3138", "Load_3139",
            "Load_3141", "Load_3142", "Load_3143", "Load_3144", "Load_3145", "Load_3146",
            "Load_3147", "Load_3148", "Load_3149", "Load_3150", "Load_3151", "Load_3152",
            "Load_3153", "Load_3154", "Load_3155", "Load_3157", "Load_3158", "Load_3159",
            "Load_3160", "Load_3161", "Load_3162"
        ]
        logger.info(f"[INIT] Defined loads: {len(self.load_names)} loads loaded.")
        
        # Validate load names against OpenDSS
        self.validate_loads()
        
        # Define system parameters
        self.KWrated = 1000
        self.load_scale = load_scale
        
        # Initialize PV systems and Batteries
        num_pv = PV_KW_ACTION_LEN  # 30 PV systems
        num_battery = BESS_KW_ACTION_LEN  # 30 Batteries
        logger.info(f"[INIT] Initializing {num_pv} PV systems and {num_battery} Batteries.")
        
        for i in range(num_pv):
            if i < len(self.ranked_buses):
                bus = self.ranked_buses[i]
                self.controller.text(f"new PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR=0 KVA=100 Pmpp=80")
                logger.debug(f"[INIT] Initialized PVSystem.PV{i+1} on Bus {bus}")
            else:
                logger.warning(f"[INIT] Not enough buses to initialize PVSystem.PV{i+1}")
        
        for i in range(num_battery):
            if i < len(self.ranked_buses):
                bus = self.ranked_buses[i]
                self.controller.text(f"new Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW=100 kVAR=0")
                logger.debug(f"[INIT] Initialized Storage.Battery{i+1} on Bus {bus}")
            else:
                logger.warning(f"[INIT] Not enough buses to initialize Storage.Battery{i+1}")
        
        # Load profiles
        self.irradiance_profile = self.load_csv(irradiance_csv_file, "Irradiance")
        self.load_profile = self.load_csv(load_profile_file, "Load Profile", skip_header=True)
        self.load_powers = self.load_csv(load_powers_file, "Load Powers", skip_header=False)
        
        # Validate load powers
        if len(self.load_powers) != len(self.load_names):
            raise ValueError("Number of load powers does not match number of load names.")
        logger.info("[INIT] Load powers and load names are consistent.")
        
        # Define action and observation spaces
        self.action_space = spaces.Box(
            low=np.array(
                [PV_KVAR_ACTION_LOW] * PV_KVAR_ACTION_LEN +
                [PV_KW_ACTION_LOW] * PV_KW_ACTION_LEN +
                [BESS_KW_ACTION_LOW] * BESS_KW_ACTION_LEN +
                [TRANSFORMER_TAPS_ACTION_LOW] * TRANSFORMER_TAPS_ACTION_LEN +
                [CAPACITOR_ACTION_LOW] * CAPACITOR_ACTION_LEN
            ),
            high=np.array(
                [PV_KVAR_ACTION_HIGH] * PV_KVAR_ACTION_LEN +
                [PV_KW_ACTION_HIGH] * PV_KW_ACTION_LEN +
                [BESS_KW_ACTION_HIGH] * BESS_KW_ACTION_LEN +
                [TRANSFORMER_TAPS_ACTION_HIGH] * TRANSFORMER_TAPS_ACTION_LEN +
                [CAPACITOR_ACTION_HIGH] * CAPACITOR_ACTION_LEN
            ),
            dtype=np.float32
        )
        logger.info(f"[INIT] Defined action space with shape: {self.action_space.shape}")
        
        # Update the number of observations based on current setup
        num_buses = len(self.ranked_buses)
        expected_observations = 915
        if num_buses != expected_observations:
            logger.warning(f"[INIT] Number of buses ({num_buses}) does not match expected observations ({expected_observations}). Please verify.")
        
        self.observation_space = spaces.Box(
            low=0.0,
            high=2.0,
            shape=(expected_observations,),
            dtype=np.float32
        )
        logger.info(f"[INIT] Defined observation space with shape: {self.observation_space.shape}")
        
        # Initialize other variables
        self.current_step = 0
        self.control_steps = 0
        self.max_control_steps = 1000  # Define as needed
        self.hourly_violations_count = []
        self.all_bus_voltages = []
    
    def load_csv(self, file_path, description, skip_header=False):
        """
        Utility method to load CSV files.
        """
        try:
            with open(file_path, 'r') as csvfile:
                reader = csv.reader(csvfile)
                if skip_header:
                    next(reader, None)  # Skip header if present
                data = [float(row[0]) for row in reader]
            logger.info(f"[LOAD] Loaded {description} with {len(data)} entries.")
            return data
        except Exception as e:
            logger.error(f"[LOAD] Failed to load {description} from {file_path}: {e}")
            raise e
    
    def validate_loads(self):
        """
        Validates that all load names exist in the OpenDSS model.
        """
        # Since 'circuit_all_load_names()' is unavailable, manual validation is limited.
        # Ensure that the load names in self.load_names match those in OpenDSS.
        # Alternatively, proceed with caution.
        logger.info("[VALIDATE] Manual validation of load names is required.")
    
    def seed(self, seed=None):
        """
        Set the seed for reproducibility.
        """
        self.seed_value = seed
        np.random.seed(seed)
        torch.manual_seed(seed)
        if seed is not None:
            torch.cuda.manual_seed_all(seed)
        logger.info(f"[SEED] Environment seeded with value: {seed}")
    
    def get_bus_names(self):
        """
        Retrieve all bus names from OpenDSS.
        """
        bus_names = self.controller.circuit_all_bus_names()
        if isinstance(bus_names, list):
            logger.info(f"[INFO] Retrieved {len(bus_names)} bus names.")
            return bus_names
        else:
            raise TypeError(f"Expected list for bus names, got {type(bus_names)}")
    
    def _take_action(self, action):
        """
        Apply the given action to the OpenDSS model.
        """
        # Split action into respective components
        pv_kvar = action[:PV_KVAR_ACTION_LEN].copy()
        pv_kw = action[PV_KVAR_ACTION_LEN:PV_KVAR_ACTION_LEN + PV_KW_ACTION_LEN].copy()
        battery_kw = action[PV_KVAR_ACTION_LEN + PV_KW_ACTION_LEN:
                            PV_KVAR_ACTION_LEN + PV_KW_ACTION_LEN + BESS_KW_ACTION_LEN].copy()
        transformer_tap = action[PV_KVAR_ACTION_LEN + PV_KW_ACTION_LEN + BESS_KW_ACTION_LEN].copy()
        capacitor_states = action[PV_KVAR_ACTION_LEN + PV_KW_ACTION_LEN + BESS_KW_ACTION_LEN + TRANSFORMER_TAPS_ACTION_LEN:].copy()
        capacitor_states = np.round(capacitor_states)
        
        # Debugging: Print actions
        logger.debug(f"[ACTION] Applying Actions:")
        logger.debug(f"  PV_kVAR Actions: {pv_kvar}")
        logger.debug(f"  PV_kW Actions: {pv_kw}")
        logger.debug(f"  Battery_kW Actions: {battery_kw}")
        logger.debug(f"  Transformer Tap Action: {transformer_tap}")
        logger.debug(f"  Capacitor States: {capacitor_states}")
        
        # Retrieve current irradiance
        irradiance = self.irradiance_profile[self.current_step % len(self.irradiance_profile)]
        logger.debug(f"[INFO] Current Irradiance: {irradiance}")
        
        # Clip PV_kW based on irradiance
        for z in range(PV_KW_ACTION_LEN):
            if pv_kw[z] > irradiance * self.KWrated:
                logger.debug(f"[CLIP] PV_kW before clipping: {pv_kw[z]} > Irradiance * KWrated: {irradiance * self.KWrated}")
                pv_kw[z] = irradiance * self.KWrated
                logger.debug(f"[CLIP] PV_kW after clipping: {pv_kw[z]}")
        
        # Clip PV_kVAR based on maximum apparent power (S_max)
        S_max = self.KWrated  # Assuming KWrated represents S_max
        q_max1 = np.sqrt(S_max**2 - np.power(pv_kw, 2))
        pv_kvar = np.clip(pv_kvar, -q_max1, q_max1)
        logger.debug(f"[ACTION] PV_kVAR after clipping: {pv_kvar}")
        
        # Apply PV and Battery actions
        for i in range(min(PV_KW_ACTION_LEN, len(self.ranked_buses))):
            bus = self.ranked_buses[i]
            scaled_pv_kw = pv_kw[i]
            self.controller.text(f"edit PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR={pv_kvar[i]} Pmpp={scaled_pv_kw}")
            logger.debug(f"[CONTROL] Edited PVSystem.PV{i+1} on Bus {bus}: kVAR={pv_kvar[i]}, Pmpp={scaled_pv_kw}")
            
            self.controller.text(f"edit Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW={battery_kw[i]} kVAR=0")
            logger.debug(f"[CONTROL] Edited Storage.Battery{i+1} on Bus {bus}: kW={battery_kw[i]}, kVAR=0")
        
        # Apply Transformer Tap action
        self.controller.text(f"edit Transformer.reg1a taps={transformer_tap}")
        logger.debug(f"[CONTROL] Edited Transformer.reg1a: taps={transformer_tap}")
        
        # Apply Capacitor States
        for i, cap_name in enumerate(self.capacitor_names):
            if i < len(capacitor_states):
                state = int(capacitor_states[i])
                self.controller.text(f"edit Capacitor.{cap_name} states={state}")
                logger.debug(f"[CONTROL] Edited Capacitor.{cap_name}: states={state}")
        
        # Solve the power flow
        self.controller.text("set controlmode=off")
        self.controller.text("solve")
        logger.debug(f"[ACTION] Actions applied and OpenDSS solved.")
    
    def step(self, action):
        """
        Perform a single step in the environment.
        """
        logger.info(f"[STEP] Step {self.current_step}: Taking action.")
        self._take_action(action)
        
        # Retrieve bus voltages
        all_bus_voltages = self.controller.circuit_all_bus_vmag_pu()
        if isinstance(all_bus_voltages, list):
            pass  # Already a list of floats
        elif isinstance(all_bus_voltages, str):
            all_bus_voltages = [float(v) for v in all_bus_voltages.split()]
        else:
            raise TypeError(f"Unexpected type for bus voltages: {type(all_bus_voltages)}")
        
        # Verify observation length matches expected
        expected_length = self.observation_space.shape[0]
        if len(all_bus_voltages) != expected_length:
            logger.error(f"[ERROR] Observation length {len(all_bus_voltages)} does not match expected {expected_length}.")
            raise ValueError(f"Observation length {len(all_bus_voltages)} does not match expected {expected_length}.")
        
        # Calculate voltage violations
        voltage_violations = sum(1 for v in all_bus_voltages if (v <= 0.95 or v >= 1.05) and v != 0)
        logger.debug(f"[OBSERVATION] Bus Voltages: {all_bus_voltages}")
        logger.info(f"[REWARD] Voltage Violations: {voltage_violations}")
        
        # Define reward
        reward = -voltage_violations
        
        # Update logs
        self.hourly_violations_count.append(voltage_violations)
        self.all_bus_voltages.append(all_bus_voltages)
        
        # Get observation
        observation = self.get_observation()
        
        # Determine if the episode is done
        termination = (self.control_steps >= self.max_control_steps) or (voltage_violations == 0)
        self.control_steps += 1
        
        if termination:
            logger.info(f"[TERMINATION] Episode terminated at step {self.control_steps} with reward {reward}.")
        
        # Return observation, reward, done, truncated, info
        return observation, reward, termination, False, {}
    
    def reset(self, *, seed=None, options=None):
        """
        Reset the environment to an initial state.
        """
        if seed is not None:
            np.random.seed(seed)
        if options is not None:
            logger.info(f"[RESET] Reset options: {options}")
        
        # Recompile OpenDSS to reset the state using the stored dss_file path
        self.controller.text(f"compile [{self.dss_file}]")
        logger.info(f"[RESET] Recompiled OpenDSS file: {self.dss_file}")
        
        # Update the current step
        self.current_step = (self.current_step + 1) % len(self.irradiance_profile)
        self.control_steps = 0
        
        # Apply load scales
        load_scales = np.random.normal(loc=self.load_profile[self.current_step], scale=0.001, size=len(self.load_names))
        logger.info(f"[RESET] Applying Load Scales for step {self.current_step}.")
        
        for load_name, load_scale in zip(self.load_names, load_scales):
            load_power = self.load_powers[self.load_names.index(load_name)]
            result = load_power * load_scale * self.load_scale
            self.controller.text(f"edit Load.{load_name} kW={result}")
            logger.debug(f"[LOAD] Edited Load.{load_name}: kW={result}")
        
        # Apply PV and Battery initial states based on irradiance
        irradiance = self.irradiance_profile[self.current_step]
        logger.info(f"[RESET] Current Irradiance: {irradiance}")
        
        for i in range(min(PV_KW_ACTION_LEN, len(self.ranked_buses))):
            bus = self.ranked_buses[i]
            fixed_power_factor = 0.9
            pv_kw = 100 * irradiance
            power_factor_angle = np.arccos(fixed_power_factor)
            pv_kvar = pv_kw * np.tan(power_factor_angle)
            battery_kw = self.load_profile[self.current_step] * 100 * self.load_scale
            self.controller.text(f"edit PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR={pv_kvar} Pmpp={pv_kw}")
            logger.debug(f"[LOAD] Edited PVSystem.PV{i+1} on Bus {bus}: kVAR={pv_kvar}, Pmpp={pv_kw}")
            
            self.controller.text(f"edit Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW={battery_kw} kVAR=0")
            logger.debug(f"[LOAD] Edited Storage.Battery{i+1} on Bus {bus}: kW={battery_kw}, kVAR=0")
        
        # Solve the power flow after resetting
        self.controller.text("set controlmode=off")
        self.controller.text("solve")
        logger.info(f"[RESET] Load and PV/Battery states updated and OpenDSS solved.")
        
        # Get initial observation
        observation = self.get_observation()
        
        # Return the initial observation
        return observation, {}
    
    def get_observation(self):
        """
        Retrieve the current observation (bus voltages).
        """
        all_bus_voltages = self.controller.circuit_all_bus_vmag_pu()
        if isinstance(all_bus_voltages, list):
            pass  # Already a list of floats
        elif isinstance(all_bus_voltages, str):
            all_bus_voltages = [float(v) for v in all_bus_voltages.split()]
        else:
            raise TypeError(f"Unexpected type for bus voltages: {type(all_bus_voltages)}")
        
        observation = np.array(all_bus_voltages).astype(np.float32).flatten()
        
        # Clip values to ensure they are within the observation space bounds
        observation = np.clip(observation, self.observation_space.low, self.observation_space.high)
        
        logger.debug(f"[GET_OBSERVATION] Retrieved Bus Voltages: {observation}")
        return observation

# ------------------------------
# 2. Register the Environment
# ------------------------------

# Define the paths to your data files
dss_path = r"C:\Program Files\OpenDSS"  # Update if different
dss_file = r"C:\Users\Alaa\Documents\OpenDSS Model\OpenDSS Model (08.13.2020)\Master.dss"
irradiance_csv_file = r"D:\Alaa_Selim\Irradiance_Profile_Santa_Clara.csv"
load_profile_file = r"D:\Alaa_Selim\LoadShape1.csv"
load_powers_file = r"C:\Users\Alaa\Documents\OpenDSS Model\OpenDSS Model (08.13.2020)\Loaddemand.csv"

def env_creator(env_config):
    """
    Environment creator for registering with RLlib.
    """
    return PowerSystemEnv(
        dss_path=dss_path,
        dss_file=dss_file,
        irradiance_csv_file=irradiance_csv_file,
        load_profile_file=load_profile_file,
        load_powers_file=load_powers_file,
        seed=SEED,
        load_scale=10 # Adjust based on your system requirements
    )

# Register the environment with RLlib
register_env("PowerSystemEnv", env_creator)

# ------------------------------
# 3. Baseline Policy Definition
# ------------------------------

def baseline_policy(action_space):
    """
    Generate a baseline action.
    For this example, the baseline action is to set all controllable elements to zero.
    Modify this function to implement more sophisticated baseline strategies if desired.
    
    Parameters:
    - action_space: The action space of the environment.
    
    Returns:
    - action: A NumPy array representing the baseline action.
    """
    # Initialize action array with zeros
    action = np.zeros(action_space.shape, dtype=np.float32)
    
    # Optionally, implement more complex baseline logic here
    # For example, keep transformer taps at default, capacitors off, etc.
    
    return action

# ------------------------------
# 4. Testing Functions
# ------------------------------

def test_sac_agent(algo, env, num_episodes=10):
    """
    Test the trained SAC agent in the environment.
    
    Parameters:
    - algo: The trained SAC algorithm instance.
    - env: The environment instance.
    - num_episodes: Number of episodes to run.
    
    Returns:
    - rewards: List of total rewards per episode.
    - lengths: List of episode lengths.
    """
    rewards = []
    lengths = []
    
    for episode in range(1, num_episodes + 1):
        try:
            obs, info = env.reset()
        except Exception as e:
            logger.error(f"[SAC TEST] Failed to reset environment for Episode {episode}: {e}")
            continue
        
        done = False
        truncated = False
        total_reward = 0.0
        steps = 0
        
        while not (done or truncated):
            try:
                # Get action from the trained policy
                action_output = algo.get_policy().compute_single_action(obs)
                
                # Extract the action from the output
                if isinstance(action_output, tuple):
                    action = action_output[0]
                else:
                    action = action_output
                
                # Ensure the action is a NumPy array
                if not isinstance(action, np.ndarray):
                    action = np.array(action)
                
                # Take the action in the environment
                obs, reward, done, truncated, info = env.step(action)
                
                # Accumulate reward and steps
                total_reward += reward
                steps += 1
            except Exception as e:
                logger.error(f"[SAC TEST] Error during Episode {episode}, Step {steps}: {e}")
                break
        
        rewards.append(total_reward)
        lengths.append(steps)
        
        print(f"SAC Agent - Episode {episode}: Total Reward = {total_reward}, Steps = {steps}")
    
    return rewards, lengths

def test_baseline_policy(env, num_episodes=10):
    """
    Test the baseline (non-control) policy in the environment.
    
    Parameters:
    - env: The environment instance.
    - num_episodes: Number of episodes to run.
    
    Returns:
    - rewards: List of total rewards per episode.
    - lengths: List of episode lengths.
    """
    rewards = []
    lengths = []
    
    for episode in range(1, num_episodes + 1):
        try:
            obs, info = env.reset()
        except Exception as e:
            logger.error(f"[BASELINE TEST] Failed to reset environment for Episode {episode}: {e}")
            continue
        
        done = False
        truncated = False
        total_reward = 0.0
        steps = 0
        
        while not (done or truncated):
            try:
                # Get baseline action
                action = baseline_policy(env.action_space)
                
                # Take the action in the environment
                obs, reward, done, truncated, info = env.step(action)
                
                # Accumulate reward and steps
                total_reward += reward
                steps += 1
            except Exception as e:
                logger.error(f"[BASELINE TEST] Error during Episode {episode}, Step {steps}: {e}")
                break
        
        rewards.append(total_reward)
        lengths.append(steps)
        
        print(f"Baseline Policy - Episode {episode}: Total Reward = {total_reward}, Steps = {steps}")
    
    return rewards, lengths

def compare_policies(sac_rewards, sac_lengths, baseline_rewards, baseline_lengths):
    """
    Compare the performance of SAC agent and baseline policy.
    
    Parameters:
    - sac_rewards: List of SAC agent rewards.
    - sac_lengths: List of SAC agent episode lengths.
    - baseline_rewards: List of baseline policy rewards.
    - baseline_lengths: List of baseline policy episode lengths.
    """
    print("\n=== Performance Comparison ===")
    
    # SAC Agent Metrics
    sac_avg_reward = np.mean(sac_rewards)
    sac_avg_length = np.mean(sac_lengths)
    
    # Baseline Policy Metrics
    baseline_avg_reward = np.mean(baseline_rewards)
    baseline_avg_length = np.mean(baseline_lengths)
    
    print(f"SAC Agent - Average Total Reward: {sac_avg_reward}, Average Episode Length: {sac_avg_length}")
    print(f"Baseline Policy - Average Total Reward: {baseline_avg_reward}, Average Episode Length: {baseline_avg_length}")
    
    # Optionally, add more comparison metrics or statistical tests here

# ------------------------------
# 5. Main Execution
# ------------------------------

def main():
    # Initialize Ray
    ray.init(ignore_reinit_error=True)
    
    # Define the path to your trained SAC checkpoint
    checkpoint_path = r"C:\Users\Alaa\ray_results\SAC\SAC_PowerSystemEnv_bd908_00000_0_lr=0.0001_2024-11-03_23-53-31\checkpoint_000257"
    
    if not os.path.exists(checkpoint_path):
        print(f"Checkpoint path does not exist: {checkpoint_path}")
        ray.shutdown()
        return
    
    # Load the trained SAC agent from the checkpoint
    try:
        algo = SAC.from_checkpoint(checkpoint_path)
        logger.info(f"[MAIN] Loaded SAC agent from checkpoint: {checkpoint_path}")
    except Exception as e:
        logger.error(f"[MAIN] Failed to load SAC agent from checkpoint: {e}")
        ray.shutdown()
        return
    
    # Create an instance of the environment
    env = env_creator({})
    
    # Define number of test episodes
    num_test_episodes = 10  # Adjust as needed
    
    # Test the SAC agent
    print("\n--- Testing SAC Agent ---")
    sac_rewards, sac_lengths = test_sac_agent(algo, env, num_test_episodes)
    
    # Test the Baseline Policy
    print("\n--- Testing Baseline Policy (No Control) ---")
    baseline_rewards, baseline_lengths = test_baseline_policy(env, num_test_episodes)
    
    # Compare the performance
    compare_policies(sac_rewards, sac_lengths, baseline_rewards, baseline_lengths)
    
    # Shutdown Ray
    ray.shutdown()

if __name__ == "__main__":
    main()


2024-11-04 00:31:20,989	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
2024-11-04 00:31:21,430	WARNING deprecation.py:50 -- DeprecationWarning: `build_tf_policy` has been deprecated. This will raise an error in the future!
2024-11-04 00:31:21,434	WARNING deprecation.py:50 -- DeprecationWarning: `build_policy_class` has been deprecated. This will raise an error in the future!
2024-11-04 00:31:49,971	INFO worker.py:1621 -- Started a local Ray instance.
2024-11-04 00:31:51,805	WARNING algorithm_config.py:656 -- Cannot create SACConfig from given `config_dict`! Property __stdout_file__ not supported.
2024-11-04 00:31:51,814	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/simple_q/` has been deprecated. Use `rllib_contrib/simple_q/` instead. This will raise an error in the future!
(pid=119420) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the f

(RolloutWorker pid=119420) OpenDSS Started successfully! 
(RolloutWorker pid=119420) OpenDSS Version 9.5.1.1 (64-bit build); License Status: Open 
(RolloutWorker pid=119420) 
(RolloutWorker pid=119420) 


(RolloutWorker pid=89720) 2024-11-04 00:31:54,715	INFO policy.py:1294 -- Policy (worker=2) running on CPU.
(RolloutWorker pid=89720) 2024-11-04 00:31:54,715	INFO torch_policy.py:185 -- Found 1 visible cuda devices.
(RolloutWorker pid=89720) 2024-11-04 00:31:54,729	INFO util.py:118 -- Using connectors:
(RolloutWorker pid=89720) 2024-11-04 00:31:54,729	INFO util.py:119 --     AgentConnectorPipeline
(RolloutWorker pid=89720)         ObsPreprocessorConnector
(RolloutWorker pid=89720)         StateBufferConnector
(RolloutWorker pid=89720)         ViewRequirementAgentConnector
(RolloutWorker pid=89720) 2024-11-04 00:31:54,729	INFO util.py:120 --     ActionConnectorPipeline
(RolloutWorker pid=89720)         ConvertToNumpyConnector
(RolloutWorker pid=89720)         NormalizeActionsConnector
(RolloutWorker pid=89720)         ImmutableActionsConnector
2024-11-04 00:31:54,834	INFO worker_set.py:297 -- Inferred observation/action spaces from remote worker (local worker has no env): {'default_polic

2024-11-04 00:31:54,957	INFO rollout_worker.py:1743 -- Built preprocessor map: {'default_policy': None}
2024-11-04 00:31:54,957	INFO rollout_worker.py:550 -- Built filter map: defaultdict(<class 'ray.rllib.utils.filter.NoFilter'>, {})
2024-11-04 00:31:54,977	WARNING util.py:68 -- Install gputil for GPU system monitoring.
INFO:__main__:[MAIN] Loaded SAC agent from checkpoint: C:\Users\Alaa\ray_results\SAC\SAC_PowerSystemEnv_bd908_00000_0_lr=0.0001_2024-11-03_23-53-31\checkpoint_000257
INFO:__main__:[SEED] Environment seeded with value: 42
INFO:__main__:[INIT] Compiled OpenDSS file: C:\Users\Alaa\Documents\OpenDSS Model\OpenDSS Model (08.13.2020)\Master.dss
INFO:__main__:[INIT] Defined capacitors: ['CAP_201', 'CAP_301']
INFO:__main__:[INFO] Retrieved 436 bus names.
INFO:__main__:[INIT] Retrieved 436 buses from OpenDSS.
INFO:__main__:[INIT] Defined loads: 194 loads loaded.
INFO:__main__:[VALIDATE] Manual validation of load names is required.
INFO:__main__:[INIT] Initializing 30 PV systems

OpenDSS Started successfully! 
OpenDSS Version 9.5.1.1 (64-bit build); License Status: Open 



--- Testing SAC Agent ---


...

INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[STEP] Step 20: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[STEP] Step 20: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[STEP] Step 20: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[STEP] Step 20: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[STEP] Step 20: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[STEP] Step 20: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[STEP] Step 20: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[STEP] Step 20: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[STEP] Step 20: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[STEP] Step 20: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[STEP] Step 20: Tak

INFO:__main__:[STEP] Step 20: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[STEP] Step 20: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[STEP] Step 20: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[STEP] Step 20: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[STEP] Step 20: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[STEP] Step 20: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[STEP] Step 20: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[STEP] Step 20: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[STEP] Step 20: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[STEP] Step 20: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[STEP] Step 20: Taking action.
INFO:__main__:[REWARD] Voltage Viol

INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[STEP] Step 20: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[STEP] Step 20: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[STEP] Step 20: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[STEP] Step 20: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[STEP] Step 20: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[STEP] Step 20: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[STEP] Step 20: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[STEP] Step 20: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[STEP] Step 20: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[STEP] Step 20: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[STEP] Step 20: Tak

INFO:__main__:[STEP] Step 20: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[STEP] Step 20: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[STEP] Step 20: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[STEP] Step 20: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[STEP] Step 20: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[STEP] Step 20: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 487
INFO:__main__:[TERMINATION] Episode terminated at step 1001 with reward -487.
(pid=80012) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future! [repeated 4x across cluster]
(RolloutWorker pid=85884) 2024-11-04 00:31:54,577	WARNING deprecation.py:50 -- DeprecationWarning: `build_tf_policy` has been deprecated. This will raise an error in the future! [repeated 4x across cluster]
(RolloutWorker pid=

Baseline Policy - Episode 10: Total Reward = -487487.0, Steps = 1001

=== Performance Comparison ===
SAC Agent - Average Total Reward: -86946.7, Average Episode Length: 1001.0
Baseline Policy - Average Total Reward: -358658.3, Average Episode Length: 1001.0
(RolloutWorker pid=85884) OpenDSS Started successfully!  [repeated 4x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(RolloutWorker pid=85884) OpenDSS Version 9.5.1.1 (64-bit build); License Status: Open  [repeated 4x across cluster]
(RolloutWorker pid=85884)  [repeated 8x across cluster]


In [2]:
# test_impala.py

import gymnasium as gym
from gymnasium import spaces
import numpy as np
from ray.tune.registry import register_env
from ray.rllib.algorithms.impala import Impala, ImpalaConfig
import pandas as pd
import torch
import warnings
import logging
import csv
import ray
from py_dss_interface import DSSDLL  # Ensure this is installed
import os

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Configure logging
logging.basicConfig(level=logging.INFO)  # Set to INFO or DEBUG as needed
logger = logging.getLogger(__name__)

# Set a seed for reproducibility
SEED = 42

# Define action space lengths
PV_KVAR_ACTION_LEN = 30          # Number of PV systems for reactive power control
PV_KW_ACTION_LEN = 30            # Number of PV systems for active power control
BESS_KW_ACTION_LEN = 30          # Number of Batteries for active power control
TRANSFORMER_TAPS_ACTION_LEN = 1  # Number of transformer taps
CAPACITOR_ACTION_LEN = 2         # Number of Capacitors (Manually defined)

# Define action value ranges
PV_KVAR_ACTION_LOW = -800
PV_KVAR_ACTION_HIGH = 800
PV_KW_ACTION_LOW = 0
PV_KW_ACTION_HIGH = 1000
BESS_KW_ACTION_LOW = 0
BESS_KW_ACTION_HIGH = 1000
TRANSFORMER_TAPS_ACTION_LOW = 0.9
TRANSFORMER_TAPS_ACTION_HIGH = 1.1
CAPACITOR_ACTION_LOW = 0
CAPACITOR_ACTION_HIGH = 1

class PowerSystemEnv(gym.Env):
    """
    Custom Environment for Power System Control using OpenDSS and Gymnasium.
    """
    metadata = {'render.modes': ['human'], 'spec.max_episode_steps': 1000}

    def __init__(self, dss_path, dss_file, irradiance_csv_file, load_profile_file, load_powers_file, seed=None, load_scale=3):
        super(PowerSystemEnv, self).__init__()
        self.seed(seed)
        
        # Store the OpenDSS file path for later use
        self.dss_file = dss_file
        
        # Initialize OpenDSS Controller
        self.controller = DSSDLL(dss_path)
        self.controller.text(f"compile [{dss_file}]")
        logger.info(f"[INIT] Compiled OpenDSS file: {dss_file}")
        
        # Manually define capacitor names
        self.capacitor_names = ["CAP_201", "CAP_301"]
        logger.info(f"[INIT] Defined capacitors: {self.capacitor_names}")
        
        # Retrieve bus names from OpenDSS
        self.ranked_buses = self.get_bus_names()
        logger.info(f"[INIT] Retrieved {len(self.ranked_buses)} buses from OpenDSS.")
        
        # Manually define load names (truncated for brevity)
        self.load_names = [
            "Load_1003", "Load_1004", "Load_1005", "Load_1006", "Load_1007", "Load_1008",
            "Load_1009", "Load_1010", "Load_1011", "Load_1012", "Load_1013", "Load_1014",
            "Load_1015", "Load_1016", "Load_1017", "Load_2002", "Load_2003", "Load_2005",
            "Load_2008", "Load_2009", "Load_2010", "Load_2011", "Load_2014", "Load_2015",
            "Load_2016", "Load_2017", "Load_2018", "Load_2020", "Load_2022", "Load_2023",
            "Load_2024", "Load_2025", "Load_2028", "Load_2029", "Load_2030", "Load_2031",
            "Load_2032", "Load_2034", "Load_2035", "Load_2037", "Load_2040", "Load_2041",
            "Load_2042", "Load_2043", "Load_2045", "Load_2046", "Load_2047", "Load_2048",
            "Load_2049", "Load_2050", "Load_2051", "Load_2052", "Load_2053", "Load_2054",
            "Load_2055", "Load_2056", "Load_2058", "Load_2059", "Load_2060",
            "Load_3002", "Load_3004", "Load_3006", "Load_3007", "Load_3009", "Load_3010",
            "Load_3011", "Load_3012", "Load_3013", "Load_3014", "Load_3016", "Load_3017",
            "Load_3018", "Load_3019", "Load_3020", "Load_3021", "Load_3023", "Load_3024",
            "Load_3025", "Load_3026", "Load_3027", "Load_3028", "Load_3029", "Load_3031",
            "Load_3032", "Load_3033", "Load_3034", "Load_3035", "Load_3036", "Load_3037",
            "Load_3038", "Load_3039", "Load_3041", "Load_3042", "Load_3043", "Load_3044",
            "Load_3045", "Load_3047", "Load_3048", "Load_3049", "Load_3050", "Load_3051",
            "Load_3052", "Load_3054", "Load_3056", "Load_3057", "Load_3058", "Load_3059",
            "Load_3060", "Load_3061", "Load_3062", "Load_3063", "Load_3064", "Load_3065",
            "Load_3066", "Load_3067", "Load_3070", "Load_3071", "Load_3072", "Load_3073",
            "Load_3074", "Load_3077", "Load_3078", "Load_3081", "Load_3083", "Load_3084",
            "Load_3085", "Load_3086", "Load_3087", "Load_3088", "Load_3089", "Load_3090",
            "Load_3091", "Load_3093", "Load_3094", "Load_3095", "Load_3096", "Load_3097",
            "Load_3098", "Load_3099", "Load_3101", "Load_3102", "Load_3103", "Load_3104",
            "Load_3105", "Load_3106", "Load_3108", "Load_3109", "Load_3110", "Load_3111",
            "Load_3112", "Load_3114", "Load_3115", "Load_3116", "Load_3117", "Load_3120",
            "Load_3121", "Load_3122", "Load_3123", "Load_3124", "Load_3125", "Load_3126",
            "Load_3127", "Load_3128", "Load_3129", "Load_3130", "Load_3131", "Load_3132",
            "Load_3134", "Load_3135", "Load_3136", "Load_3137", "Load_3138", "Load_3139",
            "Load_3141", "Load_3142", "Load_3143", "Load_3144", "Load_3145", "Load_3146",
            "Load_3147", "Load_3148", "Load_3149", "Load_3150", "Load_3151", "Load_3152",
            "Load_3153", "Load_3154", "Load_3155", "Load_3157", "Load_3158", "Load_3159",
            "Load_3160", "Load_3161", "Load_3162"
        ]
        logger.info(f"[INIT] Defined loads: {len(self.load_names)} loads loaded.")
        
        # Validate load names against OpenDSS
        self.validate_loads()
        
        # Define system parameters
        self.KWrated = 1000
        self.load_scale = load_scale
        
        # Initialize PV systems and Batteries
        num_pv = PV_KW_ACTION_LEN  # 30 PV systems
        num_battery = BESS_KW_ACTION_LEN  # 30 Batteries
        logger.info(f"[INIT] Initializing {num_pv} PV systems and {num_battery} Batteries.")
        
        for i in range(num_pv):
            if i < len(self.ranked_buses):
                bus = self.ranked_buses[i]
                self.controller.text(f"new PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR=0 KVA=100 Pmpp=80")
                logger.debug(f"[INIT] Initialized PVSystem.PV{i+1} on Bus {bus}")
            else:
                logger.warning(f"[INIT] Not enough buses to initialize PVSystem.PV{i+1}")
        
        for i in range(num_battery):
            if i < len(self.ranked_buses):
                bus = self.ranked_buses[i]
                self.controller.text(f"new Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW=100 kVAR=0")
                logger.debug(f"[INIT] Initialized Storage.Battery{i+1} on Bus {bus}")
            else:
                logger.warning(f"[INIT] Not enough buses to initialize Storage.Battery{i+1}")
        
        # Load profiles
        self.irradiance_profile = self.load_csv(irradiance_csv_file, "Irradiance")
        self.load_profile = self.load_csv(load_profile_file, "Load Profile", skip_header=True)
        self.load_powers = self.load_csv(load_powers_file, "Load Powers", skip_header=False)
        
        # Validate load powers
        if len(self.load_powers) != len(self.load_names):
            raise ValueError("Number of load powers does not match number of load names.")
        logger.info("[INIT] Load powers and load names are consistent.")
        
        # Define action and observation spaces
        self.action_space = spaces.Box(
            low=np.array(
                [PV_KVAR_ACTION_LOW] * PV_KVAR_ACTION_LEN +
                [PV_KW_ACTION_LOW] * PV_KW_ACTION_LEN +
                [BESS_KW_ACTION_LOW] * BESS_KW_ACTION_LEN +
                [TRANSFORMER_TAPS_ACTION_LOW] * TRANSFORMER_TAPS_ACTION_LEN +
                [CAPACITOR_ACTION_LOW] * CAPACITOR_ACTION_LEN
            ),
            high=np.array(
                [PV_KVAR_ACTION_HIGH] * PV_KVAR_ACTION_LEN +
                [PV_KW_ACTION_HIGH] * PV_KW_ACTION_LEN +
                [BESS_KW_ACTION_HIGH] * BESS_KW_ACTION_LEN +
                [TRANSFORMER_TAPS_ACTION_HIGH] * TRANSFORMER_TAPS_ACTION_LEN +
                [CAPACITOR_ACTION_HIGH] * CAPACITOR_ACTION_LEN
            ),
            dtype=np.float32
        )
        logger.info(f"[INIT] Defined action space with shape: {self.action_space.shape}")
        
        # Update the number of observations based on current setup
        num_buses = len(self.ranked_buses)
        expected_observations = 915
        if num_buses != expected_observations:
            logger.warning(f"[INIT] Number of buses ({num_buses}) does not match expected observations ({expected_observations}). Please verify.")
        
        self.observation_space = spaces.Box(
            low=0.0,
            high=2.0,
            shape=(expected_observations,),
            dtype=np.float32
        )
        logger.info(f"[INIT] Defined observation space with shape: {self.observation_space.shape}")
        
        # Initialize other variables
        self.current_step = 0
        self.control_steps = 0
        self.max_control_steps = 0  # Define as needed
        self.hourly_violations_count = []
        self.all_bus_voltages = []
    
    def load_csv(self, file_path, description, skip_header=False):
        """
        Utility method to load CSV files.
        """
        try:
            with open(file_path, 'r') as csvfile:
                reader = csv.reader(csvfile)
                if skip_header:
                    next(reader, None)  # Skip header if present
                data = [float(row[0]) for row in reader]
            logger.info(f"[LOAD] Loaded {description} with {len(data)} entries.")
            return data
        except Exception as e:
            logger.error(f"[LOAD] Failed to load {description} from {file_path}: {e}")
            raise e
    
    def validate_loads(self):
        """
        Validates that all load names exist in the OpenDSS model.
        """
        # Since 'circuit_all_load_names()' is unavailable, manual validation is limited.
        # Ensure that the load names in self.load_names match those in OpenDSS.
        # Alternatively, proceed with caution.
        logger.info("[VALIDATE] Manual validation of load names is required.")
    
    def seed(self, seed=None):
        """
        Set the seed for reproducibility.
        """
        self.seed_value = seed
        np.random.seed(seed)
        torch.manual_seed(seed)
        if seed is not None:
            torch.cuda.manual_seed_all(seed)
        logger.info(f"[SEED] Environment seeded with value: {seed}")
    
    def get_bus_names(self):
        """
        Retrieve all bus names from OpenDSS.
        """
        bus_names = self.controller.circuit_all_bus_names()
        if isinstance(bus_names, list):
            logger.info(f"[INFO] Retrieved {len(bus_names)} bus names.")
            return bus_names
        else:
            raise TypeError(f"Expected list for bus names, got {type(bus_names)}")
    
    def _take_action(self, action):
        """
        Apply the given action to the OpenDSS model.
        """
        # Split action into respective components
        pv_kvar = action[:PV_KVAR_ACTION_LEN].copy()
        pv_kw = action[PV_KVAR_ACTION_LEN:PV_KVAR_ACTION_LEN + PV_KW_ACTION_LEN].copy()
        battery_kw = action[PV_KVAR_ACTION_LEN + PV_KW_ACTION_LEN:
                            PV_KVAR_ACTION_LEN + PV_KW_ACTION_LEN + BESS_KW_ACTION_LEN].copy()
        transformer_tap = action[PV_KVAR_ACTION_LEN + PV_KW_ACTION_LEN + BESS_KW_ACTION_LEN].copy()
        capacitor_states = action[PV_KVAR_ACTION_LEN + PV_KW_ACTION_LEN + BESS_KW_ACTION_LEN + TRANSFORMER_TAPS_ACTION_LEN:].copy()
        capacitor_states = np.round(capacitor_states)
        
        # Debugging: Print actions
        logger.debug(f"[ACTION] Applying Actions:")
        logger.debug(f"  PV_kVAR Actions: {pv_kvar}")
        logger.debug(f"  PV_kW Actions: {pv_kw}")
        logger.debug(f"  Battery_kW Actions: {battery_kw}")
        logger.debug(f"  Transformer Tap Action: {transformer_tap}")
        logger.debug(f"  Capacitor States: {capacitor_states}")
        
        # Retrieve current irradiance
        irradiance = self.irradiance_profile[self.current_step % len(self.irradiance_profile)]
        logger.debug(f"[INFO] Current Irradiance: {irradiance}")
        
        # Clip PV_kW based on irradiance
        for z in range(PV_KW_ACTION_LEN):
            if pv_kw[z] > irradiance * self.KWrated:
                logger.debug(f"[CLIP] PV_kW before clipping: {pv_kw[z]} > Irradiance * KWrated: {irradiance * self.KWrated}")
                pv_kw[z] = irradiance * self.KWrated
                logger.debug(f"[CLIP] PV_kW after clipping: {pv_kw[z]}")
        
        # Clip PV_kVAR based on maximum apparent power (S_max)
        S_max = self.KWrated  # Assuming KWrated represents S_max
        q_max1 = np.sqrt(S_max**2 - np.power(pv_kw, 2))
        pv_kvar = np.clip(pv_kvar, -q_max1, q_max1)
        logger.debug(f"[ACTION] PV_kVAR after clipping: {pv_kvar}")
        
        # Apply PV and Battery actions
        for i in range(min(PV_KW_ACTION_LEN, len(self.ranked_buses))):
            bus = self.ranked_buses[i]
            scaled_pv_kw = pv_kw[i]
            self.controller.text(f"edit PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR={pv_kvar[i]} Pmpp={scaled_pv_kw}")
            logger.debug(f"[CONTROL] Edited PVSystem.PV{i+1} on Bus {bus}: kVAR={pv_kvar[i]}, Pmpp={scaled_pv_kw}")
            
            self.controller.text(f"edit Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW={battery_kw[i]} kVAR=0")
            logger.debug(f"[CONTROL] Edited Storage.Battery{i+1} on Bus {bus}: kW={battery_kw[i]}, kVAR=0")
        
        # Apply Transformer Tap action
        self.controller.text(f"edit Transformer.reg1a taps={transformer_tap}")
        logger.debug(f"[CONTROL] Edited Transformer.reg1a: taps={transformer_tap}")
        
        # Apply Capacitor States
        for i, cap_name in enumerate(self.capacitor_names):
            if i < len(capacitor_states):
                state = int(capacitor_states[i])
                self.controller.text(f"edit Capacitor.{cap_name} states={state}")
                logger.debug(f"[CONTROL] Edited Capacitor.{cap_name}: states={state}")
        
        # Solve the power flow
        self.controller.text("set controlmode=off")
        self.controller.text("solve")
        logger.debug(f"[ACTION] Actions applied and OpenDSS solved.")
    
    def step(self, action):
        """
        Perform a single step in the environment.
        """
        logger.info(f"[STEP] Step {self.current_step}: Taking action.")
        self._take_action(action)
        
        # Retrieve bus voltages
        all_bus_voltages = self.controller.circuit_all_bus_vmag_pu()
        if isinstance(all_bus_voltages, list):
            pass  # Already a list of floats
        elif isinstance(all_bus_voltages, str):
            all_bus_voltages = [float(v) for v in all_bus_voltages.split()]
        else:
            raise TypeError(f"Unexpected type for bus voltages: {type(all_bus_voltages)}")
        
        # Verify observation length matches expected
        expected_length = self.observation_space.shape[0]
        if len(all_bus_voltages) != expected_length:
            logger.error(f"[ERROR] Observation length {len(all_bus_voltages)} does not match expected {expected_length}.")
            raise ValueError(f"Observation length {len(all_bus_voltages)} does not match expected {expected_length}.")
        
        # Calculate voltage violations
        voltage_violations = sum(1 for v in all_bus_voltages if (v <= 0.95 or v >= 1.05) and v != 0)
        logger.debug(f"[OBSERVATION] Bus Voltages: {all_bus_voltages}")
        logger.info(f"[REWARD] Voltage Violations: {voltage_violations}")
        
        # Define reward
        reward = -voltage_violations
        
        # Update logs
        self.hourly_violations_count.append(voltage_violations)
        self.all_bus_voltages.append(all_bus_voltages)
        
        # Get observation
        observation = self.get_observation()
        
        # Determine if the episode is done
        termination = (self.control_steps >= self.max_control_steps) or (voltage_violations == 0)
        self.control_steps += 1
        
        if termination:
            logger.info(f"[TERMINATION] Episode terminated at step {self.control_steps} with reward {reward}.")
        
        # Return observation, reward, done, truncated, info
        return observation, reward, termination, False, {}
    
    def reset(self, *, seed=None, options=None):
        """
        Reset the environment to an initial state.
        """
        if seed is not None:
            np.random.seed(seed)
        if options is not None:
            logger.info(f"[RESET] Reset options: {options}")
        
        # Recompile OpenDSS to reset the state using the stored dss_file path
        self.controller.text(f"compile [{self.dss_file}]")
        #logger.info(f"[RESET] Recompiled OpenDSS file: {self.dss_file}")
        
        # Update the current step
        self.current_step = (self.current_step + 1) % len(self.irradiance_profile)
        self.control_steps = 0
        
        # Apply load scales
        load_scales = np.random.normal(loc=self.load_profile[self.current_step], scale=0.001, size=len(self.load_names))
        #logger.info(f"[RESET] Applying Load Scales for step {self.current_step}.")
        
        for load_name, load_scale in zip(self.load_names, load_scales):
            load_power = self.load_powers[self.load_names.index(load_name)]
            result = load_power * load_scale * self.load_scale
            self.controller.text(f"edit Load.{load_name} kW={result}")
            #logger.debug(f"[LOAD] Edited Load.{load_name}: kW={result}")
        
        # Apply PV and Battery initial states based on irradiance
        irradiance = self.irradiance_profile[self.current_step]
        #logger.info(f"[RESET] Current Irradiance: {irradiance}")
        
        for i in range(min(PV_KW_ACTION_LEN, len(self.ranked_buses))):
            bus = self.ranked_buses[i]
            fixed_power_factor = 0.9
            pv_kw = 100 * irradiance
            power_factor_angle = np.arccos(fixed_power_factor)
            pv_kvar = pv_kw * np.tan(power_factor_angle)
            battery_kw = self.load_profile[self.current_step] * 100 * self.load_scale
            self.controller.text(f"edit PVSystem.PV{i+1} phases=3 bus1={bus} kV=4.16 kVAR={pv_kvar} Pmpp={pv_kw}")
            #logger.debug(f"[LOAD] Edited PVSystem.PV{i+1} on Bus {bus}: kVAR={pv_kvar}, Pmpp={pv_kw}")
            
            self.controller.text(f"edit Storage.Battery{i+1} phases=3 bus1={bus} kV=4.16 kW={battery_kw} kVAR=0")
            #logger.debug(f"[LOAD] Edited Storage.Battery{i+1} on Bus {bus}: kW={battery_kw}, kVAR=0")
        
        # Solve the power flow after resetting
        self.controller.text("set controlmode=off")
        self.controller.text("solve")
        logger.info(f"[RESET] Load and PV/Battery states updated and OpenDSS solved.")
        
        # Get initial observation
        observation = self.get_observation()
        
        # Return the initial observation
        return observation, {}
    
    def get_observation(self):
        """
        Retrieve the current observation (bus voltages).
        """
        all_bus_voltages = self.controller.circuit_all_bus_vmag_pu()
        if isinstance(all_bus_voltages, list):
            pass  # Already a list of floats
        elif isinstance(all_bus_voltages, str):
            all_bus_voltages = [float(v) for v in all_bus_voltages.split()]
        else:
            raise TypeError(f"Unexpected type for bus voltages: {type(all_bus_voltages)}")
        
        observation = np.array(all_bus_voltages).astype(np.float32).flatten()
        
        # Clip values to ensure they are within the observation space bounds
        observation = np.clip(observation, self.observation_space.low, self.observation_space.high)
        
        logger.debug(f"[GET_OBSERVATION] Retrieved Bus Voltages: {observation}")
        return observation

def env_creator(env_config):
    """
    Environment creator for registering with RLlib.
    """
    # Define the paths to your data files
    dss_path = r"C:\Program Files\OpenDSS"  # Update if different
    dss_file = r"C:\Users\Alaa\Documents\OpenDSS Model\OpenDSS Model (08.13.2020)\Master.dss"
    irradiance_csv_file = r"D:\Alaa_Selim\Irradiacne_Profile_Iowa_State.csv"
    load_profile_file = r"D:\Alaa_Selim\LoadShape1.csv"
    load_powers_file = r"C:\Users\Alaa\Documents\OpenDSS Model\OpenDSS Model (08.13.2020)\Loaddemand.csv"
    
    return PowerSystemEnv(
        dss_path=dss_path,
        dss_file=dss_file,
        irradiance_csv_file=irradiance_csv_file,
        load_profile_file=load_profile_file,
        load_powers_file=load_powers_file,
        seed=SEED,
        load_scale=5  # Adjust based on your system requirements
    )

def main():
    # Initialize Ray
    ray.init(ignore_reinit_error=True)
    
    # Register the environment with RLlib
    register_env("PowerSystemEnv", env_creator)
    logger.info("Registered PowerSystemEnv with RLlib.")
    
    # Load the trained IMPALA model from the checkpoint
    checkpoint_path = r"C:\Users\Alaa\ray_results\IMPALA\IMPALA_PowerSystemEnv_0c1b1_00000_0_lr=0.0001_2024-11-07_13-42-31\checkpoint_000015"
    
    # Configure the IMPALA algorithm
    config = ImpalaConfig().environment(env="PowerSystemEnv").framework("torch")
    algo = Impala(config=config)
    
    # Restore the trained model from the checkpoint
    algo.restore(checkpoint_path)
    logger.info(f"Restored IMPALA model from checkpoint: {checkpoint_path}")
    
    # Create the environment
    env = env_creator(None)
    env.seed(SEED)
    
    # Initialize metrics
    total_rewards = 0
    voltage_violations = []
    all_bus_voltages = []
    actions_taken = []
    
    # Define the number of hours to test
    num_hours = 24
    
    for hour in range(num_hours):
        observations, info = env.reset(seed=SEED)
        done = False
        truncated = False
        
        while not done and not truncated:
            # Compute action using the trained model
            action = algo.compute_single_action(observations)
            actions_taken.append(action)
            
            # Step the environment
            observations, reward, done, truncated, info = env.step(action)
            total_rewards += reward
        
        # Collect voltage violations and bus voltages
        voltage_violations.append(env.hourly_violations_count[-1])
        all_bus_voltages.append(env.all_bus_voltages[-1])
    
    # Exporting actions taken to CSV
    actions_array = np.array(actions_taken)
    if actions_array.ndim == 1:
        # If actions are scalar
        actions_df = pd.DataFrame(actions_array, columns=['Action'])
    else:
        # If actions are vectors
        actions_df = pd.DataFrame(actions_array, columns=[f'Action {i+1}' for i in range(actions_array.shape[1])])
    
    actions_output_path = r"C:\Users\Alaa\actions_taken_iowa.csv"
    actions_df.to_csv(actions_output_path, index=False)
    logger.info(f"Actions taken saved to {actions_output_path}")
    
    # Exporting voltage violations and all bus voltages to CSV
    output_path = r"C:\Users\Alaa\voltage_violations_and_bus_voltages_PQ_L1.75.csv"
    data = {'Step': list(range(1, num_hours + 1)), 'Voltage Violations': voltage_violations}
    
    if all_bus_voltages:
        num_buses = len(all_bus_voltages[0])
        bus_voltage_columns = {f'Bus Voltage {i+1}': [voltage[i] for voltage in all_bus_voltages] for i in range(num_buses)}
        data.update(bus_voltage_columns)
    else:
        logger.warning("No bus voltages recorded.")
    
    violations_df = pd.DataFrame(data)
    violations_df.to_csv(output_path, index=False)
    logger.info(f"Voltage violations and bus voltages saved to {output_path}")
    
    # Print total rewards
    print(f"Total rewards during testing for {num_hours} hours: {total_rewards}")
    
    # Shutdown Ray
    ray.shutdown()

if __name__ == "__main__":
    main()


2024-11-07 14:58:18,554	INFO worker.py:1621 -- Started a local Ray instance.
INFO:__main__:Registered PowerSystemEnv with RLlib.
(pid=127148) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


(RolloutWorker pid=127148) OpenDSS Started successfully! 
(RolloutWorker pid=127148) OpenDSS Version 9.5.1.1 (64-bit build); License Status: Open 
(RolloutWorker pid=127148) 
(RolloutWorker pid=127148) 


(RolloutWorker pid=127148) [INIT] Number of buses (436) does not match expected observations (915). Please verify.
(RolloutWorker pid=127148) 2024-11-07 14:58:25,908	WARNING env.py:162 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
(RolloutWorker pid=127148) 2024-11-07 14:58:26,014	WARNING deprecation.py:50 -- DeprecationWarning: `LearningRateSchedule` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=127148) 2024-11-07 14:58:26,015	WARNING deprecation.py:50 -- DeprecationWarning: `EntropyCoeffSchedule` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=127148) 2024-11-07 14:58:26,020	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.fcnet.FullyConnectedNetwork` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=127148) 2024-11-07 14:58:26,020	WARNING deprecation.py:50 -- DeprecationW

OpenDSS Started successfully! 
OpenDSS Version 9.5.1.1 (64-bit build); License Status: Open 




INFO:__main__:[STEP] Step 4: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 0
INFO:__main__:[TERMINATION] Episode terminated at step 1 with reward 0.
INFO:__main__:[RESET] Load and PV/Battery states updated and OpenDSS solved.
INFO:__main__:[STEP] Step 5: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 0
INFO:__main__:[TERMINATION] Episode terminated at step 1 with reward 0.
INFO:__main__:[RESET] Load and PV/Battery states updated and OpenDSS solved.
INFO:__main__:[STEP] Step 6: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 0
INFO:__main__:[TERMINATION] Episode terminated at step 1 with reward 0.
INFO:__main__:[RESET] Load and PV/Battery states updated and OpenDSS solved.
INFO:__main__:[STEP] Step 7: Taking action.
INFO:__main__:[REWARD] Voltage Violations: 0
INFO:__main__:[TERMINATION] Episode terminated at step 1 with reward 0.
INFO:__main__:[RESET] Load and PV/Battery states updated and OpenDSS solved.
INFO:__main__:[STEP] Step 8: Taking action.
INFO

Total rewards during testing for 24 hours: -19
(RolloutWorker pid=134940) OpenDSS Started successfully! 
(RolloutWorker pid=134940) OpenDSS Version 9.5.1.1 (64-bit build); License Status: Open 
(RolloutWorker pid=134940)  [repeated 2x across cluster]


Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Users\Alaa\.conda\envs\py310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\Alaa\.conda\envs\py310\lib\site-packages\ray\rllib\execution\learner_thread.py", line 78, in run
    self.step()
  File "C:\Users\Alaa\.conda\envs\py310\lib\site-packages\ray\rllib\execution\multi_gpu_learner_thread.py", line 149, in step
    buffer_idx, released = self.ready_tower_stacks_buffer.get()
  File "C:\Users\Alaa\.conda\envs\py310\lib\site-packages\ray\rllib\execution\minibatch_buffer.py", line 55, in get
    self.buffers[self.idx] = self.inqueue.get(timeout=self.timeout)
  File "C:\Users\Alaa\.conda\envs\py310\lib\queue.py", line 179, in get
    raise Empty
_queue.Empty
Exception in thread Thread-25:
Traceback (most recent call last):
  File "C:\Users\Alaa\.conda\envs\py310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\Alaa\.conda\envs\py310\lib\site-packa

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
file_path = r'C:\Users\Alaa\voltage_violations_and_bus_voltages_PQ_L1.75.csv'
data = pd.read_csv(file_path)

# Remove the 'Voltage Violations' column and 'Step' column
voltage_data = data.drop(columns=['Step', 'Voltage Violations'])

# Convert the data into a numpy array for easier handling in the heatmap
voltage_array = voltage_data.values

# Plotting the heatmap
plt.figure(figsize=(15, 10))
sns.heatmap(voltage_array, cmap='viridis', cbar=True)
plt.title('DRL:PQ + VR + Caps')
plt.xlabel('Node Number')
plt.ylabel('Time of Day (Steps)')
# Save the figure as a PDF
plt.savefig('bus_voltages_heatmap_PQ.png', format='png', bbox_inches='tight')

# Show the plot
plt.show()

DEBUG:matplotlib:matplotlib data path: C:\Users\Alaa\.conda\envs\py310\lib\site-packages\matplotlib\mpl-data
DEBUG:matplotlib:CONFIGDIR=C:\Users\Alaa\.matplotlib
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is win32
DEBUG:matplotlib:CACHEDIR=C:\Users\Alaa\.matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from C:\Users\Alaa\.matplotlib\fontlist-v330.json
DEBUG:matplotlib.pyplot:Loaded backend module://matplotlib_inline.backend_inline version unknown.
DEBUG:matplotlib.pyplot:Loaded backend module://matplotlib_inline.backend_inline version unknown.
DEBUG:matplotlib.font_manager:findfont: Matching sans\-serif:style=normal:variant=normal:weight=normal:stretch=normal:size=10.0.
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\Users\\Alaa\\.conda\\envs\\py310\\lib\\site-packages\\matplotlib\\mpl-data\\fonts\\ttf\\STIXGeneral.ttf', name='STIXGeneral', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.0

DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\Users\\Alaa\\.conda\\envs\\py310\\lib\\site-packages\\matplotlib\\mpl-data\\fonts\\ttf\\DejaVuSerif-Italic.ttf', name='DejaVu Serif', style='italic', variant='normal', weight=400, stretch='normal', size='scalable')) = 11.05
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\Users\\Alaa\\.conda\\envs\\py310\\lib\\site-packages\\matplotlib\\mpl-data\\fonts\\ttf\\STIXSizTwoSymBol.ttf', name='STIXSizeTwoSym', style='normal', variant='normal', weight=700, stretch='normal', size='scalable')) = 10.335
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\Users\\Alaa\\.conda\\envs\\py310\\lib\\site-packages\\matplotlib\\mpl-data\\fonts\\ttf\\DejaVuSerif-Bold.ttf', name='DejaVu Serif', style='normal', variant='normal', weight=700, stretch='normal', size='scalable')) = 10.335
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\Users\\Alaa\\.conda\\envs\\py310\\lib\\site-packages\\matplo

DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\WINDOWS\\Fonts\\tahomabd.ttf', name='Tahoma', style='normal', variant='normal', weight=700, stretch='normal', size='scalable')) = 10.335
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\WINDOWS\\Fonts\\taile.ttf', name='Microsoft Tai Le', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\Windows\\Fonts\\consolab.ttf', name='Consolas', style='normal', variant='normal', weight=700, stretch='normal', size='scalable')) = 10.335
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\WINDOWS\\Fonts\\ROCKB.TTF', name='Rockwell', style='normal', variant='normal', weight=700, stretch='normal', size='scalable')) = 10.335
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\WINDOWS\\Fonts\\seguihis.ttf', name='Segoe UI Historic', style='normal', variant='normal', weight=400, stretch

DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\Windows\\Fonts\\corbeli.ttf', name='Corbel', style='italic', variant='normal', weight=400, stretch='normal', size='scalable')) = 11.05
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\Windows\\Fonts\\LBRITED.TTF', name='Lucida Bright', style='normal', variant='normal', weight=600, stretch='normal', size='scalable')) = 10.24
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\WINDOWS\\Fonts\\Candarab.ttf', name='Candara', style='normal', variant='normal', weight=700, stretch='normal', size='scalable')) = 10.335
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\WINDOWS\\Fonts\\SNAP____.TTF', name='Snap ITC', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\Windows\\Fonts\\TCCEB.TTF', name='Tw Cen MT Condensed Extra Bold', style='normal', variant='normal', weight=400,

DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\WINDOWS\\Fonts\\LFAX.TTF', name='Lucida Fax', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\Windows\\Fonts\\DUBAI-REGULAR.TTF', name='Dubai', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\Windows\\Fonts\\segoeprb.ttf', name='Segoe Print', style='normal', variant='normal', weight=700, stretch='normal', size='scalable')) = 10.335
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\WINDOWS\\Fonts\\DUBAI-MEDIUM.TTF', name='Dubai', style='normal', variant='normal', weight=500, stretch='normal', size='scalable')) = 10.145
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\WINDOWS\\Fonts\\trebucit.ttf', name='Trebuchet MS', style='italic', variant='normal', weight=400, stretch='

...

DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\Windows\\Fonts\\FRSCRIPT.TTF', name='French Script MT', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\WINDOWS\\Fonts\\LTYPEB.TTF', name='Lucida Sans Typewriter', style='normal', variant='normal', weight=600, stretch='normal', size='scalable')) = 10.24
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\Windows\\Fonts\\couri.ttf', name='Courier New', style='italic', variant='normal', weight=400, stretch='normal', size='scalable')) = 11.05
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\WINDOWS\\Fonts\\FRABKIT.TTF', name='Franklin Gothic Book', style='italic', variant='normal', weight=400, stretch='normal', size='scalable')) = 11.05
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\WINDOWS\\Fonts\\consola.ttf', name='Consolas', style='normal', variant='normal', w

DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\Windows\\Fonts\\palabi.ttf', name='Palatino Linotype', style='italic', variant='normal', weight=700, stretch='normal', size='scalable')) = 11.335
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\WINDOWS\\Fonts\\GILLUBCD.TTF', name='Gill Sans Ultra Bold Condensed', style='normal', variant='normal', weight=400, stretch='condensed', size='scalable')) = 10.25
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\Windows\\Fonts\\mmrtextb.ttf', name='Myanmar Text', style='normal', variant='normal', weight=700, stretch='normal', size='scalable')) = 10.335
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\WINDOWS\\Fonts\\HTOWERTI.TTF', name='High Tower Text', style='italic', variant='normal', weight=400, stretch='normal', size='scalable')) = 11.05
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\WINDOWS\\Fonts\\BOOKOS.TTF', name='Bookman Old Style', style='normal

DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\Windows\\Fonts\\BOD_CI.TTF', name='Bodoni MT', style='italic', variant='normal', weight=400, stretch='condensed', size='scalable')) = 11.25
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\Windows\\Fonts\\Roboto-BoldCondensed.ttf', name='Roboto', style='normal', variant='normal', weight=700, stretch='condensed', size='scalable')) = 10.535
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\Windows\\Fonts\\HARNGTON.TTF', name='Harrington', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\Windows\\Fonts\\BOD_BLAR.TTF', name='Bodoni MT', style='normal', variant='normal', weight=900, stretch='normal', size='scalable')) = 10.525
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\WINDOWS\\Fonts\\Roboto-Italic.ttf', name='Roboto', style='italic', variant='normal', weight=4

DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\WINDOWS\\Fonts\\TCM_____.TTF', name='Tw Cen MT', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\Windows\\Fonts\\BRITANIC.TTF', name='Britannic Bold', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\WINDOWS\\Fonts\\corbeli.ttf', name='Corbel', style='italic', variant='normal', weight=400, stretch='normal', size='scalable')) = 11.05
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\WINDOWS\\Fonts\\arialbd.ttf', name='Arial', style='normal', variant='normal', weight=700, stretch='normal', size='scalable')) = 6.698636363636363
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\WINDOWS\\Fonts\\COPRGTB.TTF', name='Copperplate Gothic Bold', style='normal', variant='normal', weigh

DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\Windows\\Fonts\\GARA.TTF', name='Garamond', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\WINDOWS\\Fonts\\Roboto-Regular.ttf', name='Roboto', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\WINDOWS\\Fonts\\RAGE.TTF', name='Rage Italic', style='italic', variant='normal', weight=400, stretch='normal', size='scalable')) = 11.05
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\Windows\\Fonts\\framdit.ttf', name='Franklin Gothic Medium', style='italic', variant='normal', weight=400, stretch='normal', size='scalable')) = 11.05
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='C:\\WINDOWS\\Fonts\\BOD_R.TTF', name='Bodoni MT', style='normal', variant='normal', weight=400, stretch='